In [13]:
import os
import pyspedas
from pyspedas import mms_load_fpi,tinterpol
from pytplot import tplot, get_data, store_data, tlimit, options,tplot_names
from pyspedas.mms import fpi,fgm
from pyspedas.mms.mms_orbit_plot import mms_orbit_plot
import plotly.express as px
import matplotlib.pyplot as plt
import datetime

from IPython.display import display, Latex

#Implement these later
#import mms_mvab_cl.py
#import cross_product_normal_cl.py

from pyspedas import time_string
import pandas as pd
import numpy as np


import pyspedas
import pytplot
save_to_csv = 'no'
load_from_cvs ='no'

#Get an approximate KH "lambda" period in terms of RE (here: 1 RE=6378 km — no rounding off to 6300 km)
#Use a stretch of time when many periods are present and use <T> = (T1+T2+T3...+Ti)/N and Vx from msheath.

#Get normal widths of each current sheet (CS) examined using dN = dt * (VN1 + VN2)/2 where dt is the duration in seconds, 
#VN1 is the average VN for a short time period (say 1 s or so?) right up against the CS start time, and VN2 is 
#the same average right after the end of the CS for about 1 s too. We need that CS normal width in terms of 
#km and ion inertial length (diavg). 

#Here, the average ion inertial length:
# npavg = ( npavg1 + npavg2 )/2.0   ;average plasma density (cm-3)
# omega_pi = sqrt(npavg*1e6*e^2./(mp*eps0))  ;ion angular frequency (Hz)
# diavg = (c/omega_pi)*1e-3      ;ion inertial length (km)
# c = double(2.99792458e8)      ;speed of light (m/s)
# e = 1.6022e-19        ;charge (C=As)
# mp = 1.672623*1e-27     ;proton rest mass  (kg)
# eps0 = double(8.8541878e-12)    ;epsilon (As/Vm)
#Get VAL at the edges of the CSs.







# compute size of vortices from 22:15 - 23:00

In [14]:
# times to take vx at (MSH times)
# vx_times =['2020-06-17/22:17:10','2020-06-17/22:25:34','2020-06-17/22:32:03','2020-06-17/22:38:16',
# '2020-06-17/22:44:25','2020-06-17/22:53:46','2020-06-17/23:01:31']

vx_times =['2020-06-17 22:17:29','2020-06-17 22:25:34','2020-06-17 22:32:03','2020-06-17 22:38:16',
'2020-06-17 22:44:25','2020-06-17 22:53:46','2020-06-17 23:01:31']


density_spikes_times = ['2020-06-17 22:17:05','2020-06-17 22:25:03','2020-06-17 22:31:48','2020-06-17 22:37:53',
'2020-06-17 22:44:05','2020-06-17 22:53:26','2020-06-17 23:01:11']

# #MSP times
# vx_times =['2020-06-17 22:16:36.000','2020-06-17 22:24:52.000','2020-06-17 22:31:36.000','2020-06-17 22:37:39.000',
#        '2020-06-17 22:43:53.000','2020-06-17 22:53:12.000', '2020-06-17 23:00:50']


# give a start time before the spike to begin searching the 4 MMS for that value
spike1_st = pd.to_datetime('2020-06-17/22:16:59.000').timestamp()
# give an end time around the spike to search the 4 MMS for that value
spike1_t = pd.to_datetime('2020-06-17/22:17:05.000').timestamp()
spike1 = 3.49
spike1_end = pd.to_datetime('2020-06-17/22:17:09.000').timestamp()



spike2_st = pd.to_datetime('2020-06-17/22:24:58.000').timestamp()
spike2_t = pd.to_datetime('2020-06-17/22:25:03.000').timestamp()
spike2 = 4.53
spike2_end = pd.to_datetime('2020-06-17/22:25:07.000').timestamp()



spike3_st = pd.to_datetime('2020-06-17/22:31:39.000').timestamp()
spike3_t = pd.to_datetime('2020-06-17/22:31:48.000').timestamp()
spike3 = 3.68
spike3_end = pd.to_datetime('2020-06-17/22:31:51.000').timestamp()


spike4_st = pd.to_datetime('2020-06-17/22:37:44.000').timestamp()
spike4_t = pd.to_datetime('2020-06-17/22:37:53.000').timestamp()
spike4 = 4.39
spike4_end = pd.to_datetime('2020-06-17/22:38:00.000').timestamp()


spike5_st = pd.to_datetime('2020-06-17/22:43:58.000').timestamp()
spike5_t = pd.to_datetime('2020-06-17/22:44:05.000').timestamp()
spike5 = 4.14
spike5_end = pd.to_datetime('2020-06-17/22:44:07.000').timestamp()



spike6_st = pd.to_datetime('2020-06-17/22:53:15.000').timestamp()
spike6_t = pd.to_datetime('2020-06-17/22:53:26.000').timestamp()
spike6 = 3.38
spike6_end = pd.to_datetime('2020-06-17/22:53:27.000').timestamp()


spike7_st = pd.to_datetime('2020-06-17/23:01:05.000').timestamp()
spike7_t = pd.to_datetime('2020-06-17/23:01:11.000').timestamp()
spike7 = 5.07
spike7_end = pd.to_datetime('2020-06-17/23:01:15.000').timestamp()


# spike_timerange = np.array([['2020-06-17/22:16:59.000','2020-06-17/22:17:09.000'], 
#                                     ['2020-06-17/22:24:58.000','2020-06-17/22:25:07.000'],
#                                     ['2020-06-17/22:31:39.000','2020-06-17/22:31:51.000'],
#                                     ['2020-06-17/22:37:44.000','2020-06-17/22:38:00.000'],
#                                     ['2020-06-17/22:43:58.000','2020-06-17/22:44:07.000'],
#                                     ['2020-06-17/22:53:15.000','2020-06-17/22:53:27.000'],
#                                     ['2020-06-17/23:01:05.000','2020-06-17/23:01:15.000']])

## Compute size of vortices for density spikes from 2020-06-17/23:30 - 2020-06-18/00:10

### comment out when doing a different set of spikes

In [15]:

# Exact start and stop times of CS:
# jet_candidate_timeranges = np.array([['2020-06-17/23:42:47.000','2020-06-17/23:42:59.000'], #17
#                                     ['2020-06-17/23:49:27.000','2020-06-17/23:49:49.000'],# 18
#                                     ['2020-06-17/23:57:57.599','2020-06-17/23:58:30.599']]) #19

# # 17, 18, and 19
# start and stop times of CS with padding ~2 secs on either side
# jets =['2020-06-17 23:42:40.000','2020-06-17 23:43:06.000','2020-06-17 23:49:18.000','2020-06-17 23:49:55.000',
#        '2020-06-17 23:57:50.000','2020-06-17 23:58:33.000']



# # because of the gradual increase in density in the data set(due to high time resolution) Finding spikes in density is hard
# # Therefore just identify spikes by eye:
# spike1_st = pd.to_datetime('2020-06-17/23:37:43.000').timestamp()
# spike1_end = pd.to_datetime('2020-06-17/23:37:45.000').timestamp()
# spike1 = 4.88

# spike2_st = pd.to_datetime('2020-06-17/23:43:30.000').timestamp()
# spike2_end = pd.to_datetime('2020-06-17/23:43:32.000').timestamp()
# spike2 = 4.68

# spike3_st = pd.to_datetime('2020-06-17/23:50:49.000').timestamp()
# spike3_end = pd.to_datetime('2020-06-17/23:50:51.000').timestamp()
# spike3 = 7.76

# spike4_st = pd.to_datetime('2020-06-17/23:58:34.000').timestamp()
# spike4_end = pd.to_datetime('2020-06-17/23:58:36.000').timestamp()
# spike4 = 4.576

# spike5_st = pd.to_datetime('2020-06-18/00:05:18.000').timestamp()
# spike5_end = pd.to_datetime('2020-06-18/00:05:20.000').timestamp()
# spike5 = 3.791

# spike1_t = pd.to_datetime('2020-06-17/23:37:44.000').timestamp()
# spike2_t = pd.to_datetime('2020-06-17/23:43:31.000').timestamp()
# spike3_t = pd.to_datetime('2020-06-17/23:50:50.000').timestamp()
# spike4_t = pd.to_datetime('2020-06-17/23:58:33.000').timestamp()
# spike5_t = pd.to_datetime('2020-06-18/00:05:17.000').timestamp()


# spikes = ['2020-06-17 23:37:44.000','2020-06-17 23:43:31.000','2020-06-17 23:50:50.000','2020-06-17 23:58:33.000','2020-06-18 00:05:17.000']


## Compute size of vortices for density spikes from 2020-06-17/23:00 - 2020-06-17/23:30

In [16]:
# jet_candidate_timeranges = np.array([['2020-06-17/23:03:50.000','2020-06-17/23:03:58.000'], #11
#                                      ['2020-06-17/23:08:35.700','2020-06-17/23:08:57.000'], #12
#                                      ['2020-06-17/23:10:43.000','2020-06-17/23:10:52.000'], #13
#                                      ['2020-06-17/23:17:17.299','2020-06-17/23:17:28.000']]) #14


# # # start and stop times of CS with padding ~2 secs on either side
# # jets =['2020-06-17 23:03:49.000','2020-06-17 23:03:59.000','2020-06-17 23:08:35.500','2020-06-17 23:08:59.000',
# #        '2020-06-17 23:10:42.000','2020-06-17 23:10:53.000', '2020-06-17 23:17:16','2020-06-17 23:17:29']

# jets =['2020-06-17 22:16:36.000','2020-06-17 22:24:52.000','2020-06-17 22:31:36.000','2020-06-17 22:37:39.000',
#        '2020-06-17 22:43:53.000','2020-06-17 22:53:12.000', '2020-06-17 23:00:50']


# # because of the gradual increase in density in the data set(due to high time resolution) Finding spikes in density is hard
# # Therefore just identify spikes by eye:
# spike1_st = pd.to_datetime('2020-06-17/23:37:43.000').timestamp()
# spike1_end = pd.to_datetime('2020-06-17/23:37:45.000').timestamp()
# spike1 = 4.88

# spike2_st = pd.to_datetime('2020-06-17/23:43:30.000').timestamp()
# spike2_end = pd.to_datetime('2020-06-17/23:43:32.000').timestamp()
# spike2 = 4.68

# spike3_st = pd.to_datetime('2020-06-17/23:50:49.000').timestamp()
# spike3_end = pd.to_datetime('2020-06-17/23:50:51.000').timestamp()
# spike3 = 7.76

# spike4_st = pd.to_datetime('2020-06-17/23:58:34.000').timestamp()
# spike4_end = pd.to_datetime('2020-06-17/23:58:36.000').timestamp()
# spike4 = 4.576

# spike5_st = pd.to_datetime('2020-06-18/00:05:18.000').timestamp()
# spike5_end = pd.to_datetime('2020-06-18/00:05:20.000').timestamp()
# spike5 = 3.791

# spike1_t = pd.to_datetime('2020-06-17/23:37:44.000').timestamp()
# spike2_t = pd.to_datetime('2020-06-17/23:43:31.000').timestamp()
# spike3_t = pd.to_datetime('2020-06-17/23:50:50.000').timestamp()
# spike4_t = pd.to_datetime('2020-06-17/23:58:33.000').timestamp()
# spike5_t = pd.to_datetime('2020-06-18/00:05:17.000').timestamp()

In [17]:
# Constants
c = 2.99792458e8     #speed of light (m/s)
q = 1.6022e-19        #charge (C=As)
mp = 1.672623*1e-27     #proton rest mass  (kg)
eps0 = 8.8541878e-12    #epsilon (As/Vm)
RE = 6378 # km

In [18]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return int(idx)

# MVAB

In [19]:
from numpy import linalg as LA
# #########################
# #GET MAGNETIC FIELD DATA#
# #########################
def mms_mvab(m1,m2,mag_arr_str):
    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)
    # ##########################################################
    # Check for NaN values
    NaNs = mag_arr.isnull().values.any()
    if NaNs == True:
        print,'Nans were found: '
        print, mag_arr.isnull().values.any()
        print,'Replacing with average of surrounding data pts '
        # Indices of NaNs in each component
        NaNs_Bx = np.argwhere(np.isnan(bx))
        NaNs_By = np.argwhere(np.isnan(by))
        NaNs_Bz = np.argwhere(np.isnan(bz))

        # If there are NaNs, replace with the average of surrounding couple of data points
        for i in range(len(NaNs_Bx)):
            bx[NaNs_Bx[i]] = np.nanmean(bx[NaNs_Bx[i-3]:NaNs_Bx[i+3]])
            by[NaNs_By[i]] = np.nanmean(by[NaNs_By[i-3]:NaNs_By[i+3]])
            bz[NaNs_Bz[i]] = np.nanmean(bz[NaNs_Bz[i-3]:NaNs_Bz[i+3]])                           
    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))
    n1 = mag_arr.date[0]
    n2 = mag_arr.date[len(mag_arr.date)-1]
    if m1 < n1:
        print('WARNING!!! start time of current sheet =',m1, 'start time of B-field data =', n1)
    if m1 > n2:
        print('WARNING!!! start time of current sheet =',m1,'end time of B-field data =',n2)
    if m2 > n2:
        print('WARNING!!! end time of current sheet =',m2,'end time of B-field data =',n2)
    if m2 < n1:
        print('WARNING!!! end time of current sheet =',m2,'start time of B-field data =',n1)
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)
    bx = bx[v1:v2]
    by = by[v1:v2]
    bz = bz[v1:v2]
    
    
    # create the minimum variance matrix. 
    mm = np.zeros([3, 3])
    bxa = np.mean(bx)
    bya = np.mean(by)
    bza = np.mean(bz)
    mm[0,0] = np.mean(bx*bx) - bxa*bxa
    mm[1,0] = np.mean(bx*by) - bxa*bya
    mm[2,0] = np.mean(bx*bz) - bxa*bza
    mm[0,1] = mm[1,0]
    mm[1,1] = np.mean(by*by) - bya*bya
    mm[2,1] = np.mean(by*bz) - bya*bza
    mm[0,2] = mm[2,0]
    mm[1,2] = mm[2,1]
    mm[2,2] = np.mean(bz*bz) - bza*bza
    
    # The 3 eigenvectors represent directions of maximum, intermediate, and minimum variance of the field
    # component along each vector. 
    
    # The corresponding eigenvalues (λ) represent the actual variances in those field components & are non-negative 
    
    
    # The eigenvector x3 (corresponding to the smallest eigenvalue λ3) is used as the estimator for 
    # the vector normal to the current sheet.
    
    # λ3 represents the variance of the magnetic field component along the estimated normal. 
    
    # The eigenvectors x1 and x2, corresponding to maximum and intermediate variance
    # are tangential to the transition layer
    
    
    
    # eigen vectors should be in GSE coordinate system right?
    eigenvalues, eigenvectors = LA.eig(mm)    
    idx = np.argsort(eigenvalues)
    E_vals_sorted = eigenvalues[idx]
    E_vec_sorted = eigenvectors[:,idx]

    return E_vals_sorted,E_vec_sorted


# Cross Product Normal

In [20]:
# Below are some lines of IDL code to get the cross-product normal.
def Cross_product_normal(E_vals,E_vec,m1,m2,mag_arr_str): 
    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)
    
    # lv,mv,nv are in the same coordinate system as the eigenvectors (GSE)
    nv = E_vec[:,0] # take B1 x B2/|B1 x B2|
    lv = E_vec[:,2]
    mv = np.cross(nv,lv)
    rat = E_vals[1]/E_vals[0]
    lxm = lv[0]
    lym = lv[1]
    lzm = lv[2]
    mxm = mv[0]
    mym = mv[1]
    mzm = mv[2]
    nxm = nv[0]
    nym = nv[1]
    nzm = nv[2]
    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))
    # search the B-field array date column for m1
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)
    
    # pick a time delta 
    dtb = pd.Timedelta(hours = 0, minutes = 0, seconds = 2)
    v1a = mag_arr.date.searchsorted(m1 - dtb)
    v2a = mag_arr.date.searchsorted(m2 + dtb)
    bv1x = bx[v1a:v1]
    bv1y = by[v1a:v1]
    bv1z = bz[v1a:v1]
    bv2x = bx[v2:v2a]
    bv2y = by[v2:v2a]
    bv2z = bz[v2:v2a]
    bv1avg = np.array([np.mean(bv1x),np.mean(bv1y),np.mean(bv1z)])
    bv2avg = np.array([np.mean(bv2x),np.mean(bv2y),np.mean(bv2z)])
    bmag1 = np.sqrt(np.sum(np.dot(bv1avg,bv1avg)))
    # original bmag2 = np.sqrt(np.sum(np.multiply(bv2avg,bv2avg)))
    bmag2 = np.sqrt(np.sum(np.dot(bv2avg,bv2avg)))
    normal = np.cross(bv1avg,bv2avg)
    normag = np.sqrt(np.sum(np.dot(normal,normal)))
    nc = normal/normag  # N-direction
    # print('nc',nc)
    # print('nc[1]',nc[1])
    # #Angle between cross-product & MVAB normals
    #n12ang = np.arccos(np.sum(nv*nc))*180./!dpi
    
    #not sure if we need this part
    n12ang = np.arccos(np.sum(nv*nc))*180
    # if n12ang >= 90.0:
    #     nc = -nc
# # flip $N_{GSE}$ to $-N_{GSE}$ if needed for all events with $N_{y}>0$
    if nc[1] > 0:
        nyc = -nc[1]
    else:
        nyc = nc[1]
        
    mvec = np.cross(nc,E_vec[:,2])
    mmag = np.sqrt(np.sum(np.dot(mvec,mvec)))
    mc = mvec/mmag    # M-direction
    lc = np.cross(mc,nc)  # L-direction
    # angle = np.arccos(np.sum(np.multiply(E_vec[:,2],lc)))*180
    # Angle between cross-product & MVAB normals
    #n12ang = np.arccos(np.sum(np.dot(nv,nc)))*180
    #print('n12ang:',n12ang)
    lxc = lc[0]
    lyc = lc[1]
    lzc = lc[2]
    mxc = mc[0]
    myc = mc[1]
    mzc = mc[2]
    nxc = nc[0]
    nyc = nc[1]
    
    nzc = nc[2]
    nang = n12ang
    
    # display(Latex(f'$n_c \cdot l_c =$ {np.dot(nc,lc):.2f}')) 
    # display(Latex(f'$m_c \cdot l_c =$ {np.dot(mc,lc):.2f}'))
    # display(Latex(f'$n_c \cdot m_c =$ {np.dot(nc,mc):.2f}'))

#     display(Latex(f"$L_x =$ {lxc:.2f}"))
#     display(Latex(f"$L_y =$ {lyc:.2f}"))
#     display(Latex(f"$L_z =$ {lzc:.2f}"))
    
#     display(Latex(f"$L_xm =$ {lxm:.2f}"))
#     display(Latex(f"$L_ym =$ {lym:.2f}"))
#     display(Latex(f"$L_zm =$ {lzm:.2f}"))
    return lc,mc,nc,lv,mv,nv,mag_arr['date'][v1a],mag_arr['date'][v2a]

# Load data from pySPEDAS

In [21]:
if load_from_cvs == 'no':
    ion_vars = pyspedas.mms.mms_load_fpi(trange=['2020-06-17/21:00:00','2020-06-18/01:30:00'],
                                     probe=['1','2','3','4'],data_rate='brst',datatype='dis-moms',
                                     center_measurement=True,time_clip=True)

    fgm_vars = pyspedas.mms.mms_load_fgm(trange=['2020-06-17/21:00:00','2020-06-18/01:30:00'],
                                     probe=['1','2','3','4'], data_rate = 'brst',time_clip=True)

    mec_vars = pyspedas.mms.mec(trange=['2020-06-17/21:00:00','2020-06-18/01:30:00'],
                            probe=['1','2','3','4'], data_rate = 'brst')

    ni_mms1 = 'mms1_dis_numberdensity_brst'
    times_ni_mms1, data_ni_mms1 = get_data(ni_mms1)
    Vi_mms1 = 'mms1_dis_bulkv_gse_brst'
    times_vi_mms1, data_vi_mms1 = get_data(Vi_mms1)
    b_field_mms1 = 'mms1_fgm_b_gse_brst_l2'
    times_b_mms1, data_b_mms1 = get_data(b_field_mms1)
    
    ni_mms2 = 'mms2_dis_numberdensity_brst'
    times_ni_mms2, data_ni_mms2 = get_data(ni_mms2)
    Vi_mms2 = 'mms2_dis_bulkv_gse_brst'
    times_vi_mms2, data_vi_mms2 = get_data(Vi_mms2)
    b_field_mms2 = 'mms2_fgm_b_gse_brst_l2'
    times_b_mms2, data_b_mms2 = get_data(b_field_mms2)
    
    ni_mms3 = 'mms3_dis_numberdensity_brst'
    times_ni_mms3, data_ni_mms3 = get_data(ni_mms3)
    Vi_mms3 = 'mms3_dis_bulkv_gse_brst'
    times_vi_mms3, data_vi_mms3 = get_data(Vi_mms3)
    b_field_mms3 = 'mms3_fgm_b_gse_brst_l2'
    times_b_mms3, data_b_mms3 = get_data(b_field_mms3)
    
    ni_mms4 = 'mms4_dis_numberdensity_brst'
    times_ni_mms4, data_ni_mms4 = get_data(ni_mms4)
    Vi_mms4 = 'mms4_dis_bulkv_gse_brst'
    times_vi_mms4, data_vi_mms4 = get_data(Vi_mms4)
    b_field_mms4 = 'mms4_fgm_b_gse_brst_l2'
    times_b_mms4, data_b_mms4 = get_data(b_field_mms4)
    
    mec_mms1 = 'mms1_mec_r_gse'
    times_mec_mms1, data_mec_mms1 = get_data(mec_mms1)
    
    
    mec_mms2 = 'mms2_mec_r_gse'
    times_mec_mms2, data_mec_mms2 = get_data(mec_mms2)
    
    mec_mms3 = 'mms3_mec_r_gse'
    times_mec_mms3, data_mec_mms3 = get_data(mec_mms3)
    
    mec_mms4 = 'mms4_mec_r_gse'
    times_mec_mms4, data_mec_mms4 = get_data(mec_mms4)
    
    tperp_mms1 = 'mms1_dis_tempperp_brst'
    times_tperp_mms1, data_tperp_mms1 = get_data(tperp_mms1)
    tperp_mms2 = 'mms2_dis_tempperp_brst'
    times_tperp_mms2, data_tperp_mms2 = get_data(tperp_mms2)
    tperp_mms3 = 'mms3_dis_tempperp_brst'
    times_tperp_mms3, data_tperp_mms3 = get_data(tperp_mms3)
    tperp_mms4 = 'mms4_dis_tempperp_brst'
    times_tperp_mms4, data_tperp_mms4 = get_data(tperp_mms4)
    
    tpara_mms1 = 'mms1_dis_temppara_brst'
    times_tpara_mms1, data_tpara_mms1 = get_data(tpara_mms1)
    tpara_mms2 = 'mms2_dis_temppara_brst'
    times_tpara_mms2, data_tpara_mms2 = get_data(tpara_mms2)
    tpara_mms3 = 'mms3_dis_temppara_brst'
    times_tpara_mms3, data_tpara_mms3 = get_data(tpara_mms3)
    tpara_mms4 = 'mms4_dis_temppara_brst'
    times_tpara_mms4, data_tpara_mms4 = get_data(tpara_mms4)
    
    
    
    
    ave_ti_mms1 =[]
    ave_ti_mms2 =[]
    ave_ti_mms3 =[]
    ave_ti_mms4 =[]
    for i in range(len(data_tpara_mms1)):
        ave_ti_mms1.append(np.mean([data_tpara_mms1[i],data_tperp_mms1[i]]))
        
    for i in range(len(data_tpara_mms2)):
        ave_ti_mms2.append(np.mean([data_tpara_mms2[i],data_tperp_mms2[i]]))  
        
    for i in range(len(data_tpara_mms3)):
        ave_ti_mms3.append(np.mean([data_tpara_mms3[i],data_tperp_mms3[i]]))
    
    for i in range(len(data_tpara_mms4)):
        ave_ti_mms4.append(np.mean([data_tpara_mms4[i],data_tperp_mms4[i]]))
        
    MMS1_ni = pd.DataFrame({"times": times_ni_mms1, "ni": data_ni_mms1})
    MMS1_ni['date'] = pd.to_datetime(MMS1_ni['times'],unit='s')
    MMS1_Vi = pd.DataFrame({"times": times_vi_mms1, "Vix": data_vi_mms1[:,0],"Viy": data_vi_mms1[:,1], "Viz": data_vi_mms1[:,2]})
    MMS1_Vi['date'] = pd.to_datetime(MMS1_Vi['times'],unit='s')
    MMS1_B_gse = pd.DataFrame({"times": times_b_mms1, "Bx": data_b_mms1[:,0],"By": data_b_mms1[:,1], "Bz": data_b_mms1[:,2],"Bmag": data_b_mms1[:,3]})
    MMS1_B_gse['date'] = pd.to_datetime(MMS1_B_gse['times'],unit='s')
    MMS1_ti = pd.DataFrame({"times": times_tperp_mms1, "ti": ave_ti_mms1})
    MMS1_ti['date'] = pd.to_datetime(MMS1_ti['times'],unit='s')
    
    MMS2_ni = pd.DataFrame({"times": times_ni_mms2, "ni": data_ni_mms2})
    MMS2_ni['date'] = pd.to_datetime(MMS2_ni['times'],unit='s')
    MMS2_Vi = pd.DataFrame({"times": times_vi_mms2, "Vix": data_vi_mms2[:,0],"Viy": data_vi_mms2[:,1], "Viz": data_vi_mms2[:,2]})
    MMS2_Vi['date'] = pd.to_datetime(MMS2_Vi['times'],unit='s')
    MMS2_B_gse = pd.DataFrame({"times": times_b_mms2, "Bx": data_b_mms2[:,0],"By": data_b_mms2[:,1], "Bz": data_b_mms2[:,2],"Bmag": data_b_mms2[:,3]})
    MMS2_B_gse['date'] = pd.to_datetime(MMS2_B_gse['times'],unit='s')
    MMS2_ti = pd.DataFrame({"times": times_tperp_mms2, "ti": ave_ti_mms2})
    MMS2_ti['date'] = pd.to_datetime(MMS2_ti['times'],unit='s')
    
    MMS3_ni = pd.DataFrame({"times": times_ni_mms3, "ni": data_ni_mms3})
    MMS3_ni['date'] = pd.to_datetime(MMS3_ni['times'],unit='s')
    MMS3_Vi = pd.DataFrame({"times": times_vi_mms3, "Vix": data_vi_mms3[:,0],"Viy": data_vi_mms3[:,1], "Viz": data_vi_mms3[:,2]})
    MMS3_Vi['date'] = pd.to_datetime(MMS3_Vi['times'],unit='s')
    MMS3_B_gse = pd.DataFrame({"times": times_b_mms3, "Bx": data_b_mms3[:,0],"By": data_b_mms3[:,1], "Bz": data_b_mms3[:,2],"Bmag": data_b_mms3[:,3]})
    MMS3_B_gse['date'] = pd.to_datetime(MMS3_B_gse['times'],unit='s')
    MMS3_ti = pd.DataFrame({"times": times_tperp_mms3, "ti": ave_ti_mms3})
    MMS3_ti['date'] = pd.to_datetime(MMS3_ti['times'],unit='s')
    
    MMS4_ni = pd.DataFrame({"times": times_ni_mms4, "ni": data_ni_mms4})
    MMS4_ni['date'] = pd.to_datetime(MMS4_ni['times'],unit='s')
    MMS4_Vi = pd.DataFrame({"times": times_vi_mms4, "Vix": data_vi_mms4[:,0],"Viy": data_vi_mms4[:,1], "Viz": data_vi_mms4[:,2]})
    MMS4_Vi['date'] = pd.to_datetime(MMS4_Vi['times'],unit='s')
    MMS4_B_gse = pd.DataFrame({"times": times_b_mms4, "Bx": data_b_mms4[:,0],"By": data_b_mms4[:,1], "Bz": data_b_mms4[:,2],"Bmag": data_b_mms4[:,3]})
    MMS4_B_gse['date'] = pd.to_datetime(MMS4_B_gse['times'],unit='s')
    MMS4_ti = pd.DataFrame({"times": times_tperp_mms4, "ti": ave_ti_mms4})
    MMS4_ti['date'] = pd.to_datetime(MMS4_ti['times'],unit='s')
    
    MMS1_r_gse = pd.DataFrame({"times": times_mec_mms1, "x": data_mec_mms1[:,0],"y": data_mec_mms1[:,1], "z": data_mec_mms1[:,2]})
    MMS1_r_gse['date'] = pd.to_datetime(MMS1_r_gse['times'],unit='s')
    
    MMS2_r_gse = pd.DataFrame({"times": times_mec_mms2, "x": data_mec_mms2[:,0],"y": data_mec_mms2[:,1], "z": data_mec_mms2[:,2]})
    MMS2_r_gse['date'] = pd.to_datetime(MMS2_r_gse['times'],unit='s')
    
    MMS3_r_gse = pd.DataFrame({"times": times_mec_mms3, "x": data_mec_mms3[:,0],"y": data_mec_mms3[:,1], "z": data_mec_mms3[:,2]})
    MMS3_r_gse['date'] = pd.to_datetime(MMS3_r_gse['times'],unit='s')
    
    MMS4_r_gse = pd.DataFrame({"times": times_mec_mms4, "x": data_mec_mms4[:,0],"y": data_mec_mms4[:,1], "z": data_mec_mms4[:,2]})
    MMS4_r_gse['date'] = pd.to_datetime(MMS4_r_gse['times'],unit='s')
    
    
    if save_to_csv == 'yes':
        MMS1_B_gse.to_csv('MMS1_B_gse.csv', index=False)  
        MMS1_ni.to_csv('MMS1_ni.csv', index=False)  
        MMS1_Vi.to_csv('MMS1_Vi.csv', index=False)  
        MMS2_B_gse.to_csv('MMS2_B_gse.csv', index=False)  
        MMS2_ni.to_csv('MMS2_ni.csv', index=False)  
        MMS2_Vi.to_csv('MMS2_Vi.csv', index=False)  
        
        MMS3_B_gse.to_csv('MMS3_B_gse.csv', index=False)  
        MMS3_ni.to_csv('MMS3_ni.csv', index=False)  
        MMS3_Vi.to_csv('MMS3_Vi.csv', index=False)
        MMS4_B_gse.to_csv('MMS4_B_gse.csv', index=False)  
        MMS4_ni.to_csv('MMS4_ni.csv', index=False)  
        MMS4_Vi.to_csv('MMS4_Vi.csv', index=False)
          

19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617205913_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617210143_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617210413_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617210643_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617210913_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617211133_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617211403_v3.4.0.cdf
19-Feb-25 23:53:12: Loading pydata/mms1/fpi/brst/l2/dis-moms/2020/06/17/mms1_fpi_brst_l2_dis-moms_20200617211633_v3.4.0.cdf
19-Feb-2

In [22]:
#tplot_names()

## Get pyspedas data into data structures

## Save to CSV

In [23]:
# MMS1_ni = pd.DataFrame({"times": times_ni_mms1, "ni": data_ni_mms1})
# MMS1_ni['date'] = pd.to_datetime(MMS1_ni['times'],unit='s')
# MMS1_Vi = pd.DataFrame({"times": times_vi_mms1, "Vix": data_vi_mms1[:,0],"Viy": data_vi_mms1[:,1], "Viz": data_vi_mms1[:,2]})
# MMS1_Vi['date'] = pd.to_datetime(MMS1_Vi['times'],unit='s')
# MMS1_B_gse = pd.DataFrame({"times": times_b_mms1, "Bx": data_b_mms1[:,0],"By": data_b_mms1[:,1], "Bz": data_b_mms1[:,2],"Bmag": data_b_mms1[:,3]})
# MMS1_B_gse['date'] = pd.to_datetime(MMS1_B_gse['times'],unit='s')
# MMS1_ti = pd.DataFrame({"times": times_tperp_mms1, "ti": ave_ti_mms1})
# MMS1_ti['date'] = pd.to_datetime(MMS1_ti['times'],unit='s')

# MMS2_ni = pd.DataFrame({"times": times_ni_mms2, "ni": data_ni_mms2})
# MMS2_ni['date'] = pd.to_datetime(MMS2_ni['times'],unit='s')
# MMS2_Vi = pd.DataFrame({"times": times_vi_mms2, "Vix": data_vi_mms2[:,0],"Viy": data_vi_mms2[:,1], "Viz": data_vi_mms2[:,2]})
# MMS2_Vi['date'] = pd.to_datetime(MMS2_Vi['times'],unit='s')
# MMS2_B_gse = pd.DataFrame({"times": times_b_mms2, "Bx": data_b_mms2[:,0],"By": data_b_mms2[:,1], "Bz": data_b_mms2[:,2],"Bmag": data_b_mms2[:,3]})
# MMS2_B_gse['date'] = pd.to_datetime(MMS2_B_gse['times'],unit='s')
# MMS2_ti = pd.DataFrame({"times": times_tperp_mms2, "ti": ave_ti_mms2})
# MMS2_ti['date'] = pd.to_datetime(MMS2_ti['times'],unit='s')

# MMS3_ni = pd.DataFrame({"times": times_ni_mms3, "ni": data_ni_mms3})
# MMS3_ni['date'] = pd.to_datetime(MMS3_ni['times'],unit='s')
# MMS3_Vi = pd.DataFrame({"times": times_vi_mms3, "Vix": data_vi_mms3[:,0],"Viy": data_vi_mms3[:,1], "Viz": data_vi_mms3[:,2]})
# MMS3_Vi['date'] = pd.to_datetime(MMS3_Vi['times'],unit='s')
# MMS3_B_gse = pd.DataFrame({"times": times_b_mms3, "Bx": data_b_mms3[:,0],"By": data_b_mms3[:,1], "Bz": data_b_mms3[:,2],"Bmag": data_b_mms3[:,3]})
# MMS3_B_gse['date'] = pd.to_datetime(MMS3_B_gse['times'],unit='s')
# MMS3_ti = pd.DataFrame({"times": times_tperp_mms3, "ti": ave_ti_mms3})
# MMS3_ti['date'] = pd.to_datetime(MMS3_ti['times'],unit='s')

# MMS4_ni = pd.DataFrame({"times": times_ni_mms4, "ni": data_ni_mms4})
# MMS4_ni['date'] = pd.to_datetime(MMS4_ni['times'],unit='s')
# MMS4_Vi = pd.DataFrame({"times": times_vi_mms4, "Vix": data_vi_mms4[:,0],"Viy": data_vi_mms4[:,1], "Viz": data_vi_mms4[:,2]})
# MMS4_Vi['date'] = pd.to_datetime(MMS4_Vi['times'],unit='s')
# MMS4_B_gse = pd.DataFrame({"times": times_b_mms4, "Bx": data_b_mms4[:,0],"By": data_b_mms4[:,1], "Bz": data_b_mms4[:,2],"Bmag": data_b_mms4[:,3]})
# MMS4_B_gse['date'] = pd.to_datetime(MMS4_B_gse['times'],unit='s')
# MMS4_ti = pd.DataFrame({"times": times_tperp_mms4, "ti": ave_ti_mms4})
# MMS4_ti['date'] = pd.to_datetime(MMS4_ti['times'],unit='s')

# MMS1_r_gse = pd.DataFrame({"times": times_mec_mms1, "x": data_mec_mms1[:,0],"y": data_mec_mms1[:,1], "z": data_mec_mms1[:,2]})
# MMS1_r_gse['date'] = pd.to_datetime(MMS1_r_gse['times'],unit='s')

# MMS2_r_gse = pd.DataFrame({"times": times_mec_mms2, "x": data_mec_mms2[:,0],"y": data_mec_mms2[:,1], "z": data_mec_mms2[:,2]})
# MMS2_r_gse['date'] = pd.to_datetime(MMS2_r_gse['times'],unit='s')

# MMS3_r_gse = pd.DataFrame({"times": times_mec_mms3, "x": data_mec_mms3[:,0],"y": data_mec_mms3[:,1], "z": data_mec_mms3[:,2]})
# MMS3_r_gse['date'] = pd.to_datetime(MMS3_r_gse['times'],unit='s')

# MMS4_r_gse = pd.DataFrame({"times": times_mec_mms4, "x": data_mec_mms4[:,0],"y": data_mec_mms4[:,1], "z": data_mec_mms4[:,2]})
# MMS4_r_gse['date'] = pd.to_datetime(MMS4_r_gse['times'],unit='s')


# if save_to_csv == 'yes':
#     MMS1_B_gse.to_csv('MMS1_B_gse.csv', index=False)  
#     MMS1_ni.to_csv('MMS1_ni.csv', index=False)  
#     MMS1_Vi.to_csv('MMS1_Vi.csv', index=False)  
#     MMS2_B_gse.to_csv('MMS2_B_gse.csv', index=False)  
#     MMS2_ni.to_csv('MMS2_ni.csv', index=False)  
#     MMS2_Vi.to_csv('MMS2_Vi.csv', index=False)  
    
#     MMS3_B_gse.to_csv('MMS3_B_gse.csv', index=False)  
#     MMS3_ni.to_csv('MMS3_ni.csv', index=False)  
#     MMS3_Vi.to_csv('MMS3_Vi.csv', index=False)
#     MMS4_B_gse.to_csv('MMS4_B_gse.csv', index=False)  
#     MMS4_ni.to_csv('MMS4_ni.csv', index=False)  
#     MMS4_Vi.to_csv('MMS4_Vi.csv', index=False)


# Plot data

In [24]:
# %matplotlib widget
# figure, axis = plt.subplots(5, 1,  figsize=(10, 5),sharex=True) #constrained_layout=True
# #plt.subplots_adjust(wspace=0, hspace=0)
# # axis[0].plot(np.array(time_string(timeB)).astype("datetime64[ns]"),dataB[:,:])
# axis[0].plot(np.array(time_string(times_ni_mms1)).astype("datetime64[ns]"),data_ni_mms1, 'k',label='MMS1')
# axis[0].plot(np.array(time_string(times_ni_mms2)).astype("datetime64[ns]"),data_ni_mms2, 'green',label='MMS2')
# axis[0].plot(np.array(time_string(times_ni_mms3)).astype("datetime64[ns]"),data_ni_mms3, 'darkturquoise',label='MMS3')
# axis[0].plot(np.array(time_string(times_ni_mms4)).astype("datetime64[ns]"),data_ni_mms4, 'purple',label='MMS4')
# axis[0].legend(fontsize=14)
# axis[0].set_ylabel('$n_i$ $[cm^{-3}]$')
# #axis[0].set_ylim([0,5])


# axis[1].plot(np.array(time_string(times_tperp_mms1)).astype("datetime64[ns]"),ave_ti_mms1, 'k',label='MMS1')
# axis[1].plot(np.array(time_string(times_tperp_mms2)).astype("datetime64[ns]"),ave_ti_mms2, 'green',label='MMS2')
# axis[1].plot(np.array(time_string(times_tperp_mms3)).astype("datetime64[ns]"),ave_ti_mms3, 'darkturquoise',label='MMS3')
# axis[1].plot(np.array(time_string(times_tperp_mms4)).astype("datetime64[ns]"),ave_ti_mms4, 'purple',label='MMS4')
# axis[1].legend(fontsize=14)
# axis[1].set_ylabel('$t_i$ $[eV]$')
# #axis[1].set_ylim([0,5])




# axis[2].plot(np.array(time_string(times_vi_mms1)).astype("datetime64[ns]"),data_vi_mms1[:,0], 'k', label='MMS1')
# axis[2].plot(np.array(time_string(times_vi_mms2)).astype("datetime64[ns]"),data_vi_mms2[:,0], 'green', label='MMS2')
# axis[2].plot(np.array(time_string(times_vi_mms3)).astype("datetime64[ns]"),data_vi_mms3[:,0], 'darkturquoise', label='MMS3')
# axis[2].plot(np.array(time_string(times_vi_mms4)).astype("datetime64[ns]"),data_vi_mms4[:,0], 'purple', label='MMS4')
# axis[2].legend(fontsize=14)
# axis[2].set_ylabel('$v_x$ [km/s]')



# axis[3].plot(np.array(time_string(times_vi_mms1)).astype("datetime64[ns]"),data_vi_mms1[:,1], 'k', label='MMS1')
# axis[3].plot(np.array(time_string(times_vi_mms2)).astype("datetime64[ns]"),data_vi_mms2[:,1], 'green', label='MMS2')
# axis[3].plot(np.array(time_string(times_vi_mms3)).astype("datetime64[ns]"),data_vi_mms3[:,1], 'darkturquoise', label='MMS3')
# axis[3].plot(np.array(time_string(times_vi_mms4)).astype("datetime64[ns]"),data_vi_mms4[:,1], 'purple', label='MMS4')
# axis[3].legend(fontsize=14)
# axis[3].set_ylabel('$v_y$ [km/s]')

# axis[4].plot(np.array(time_string(times_vi_mms1)).astype("datetime64[ns]"),data_vi_mms1[:,2], 'k', label='MMS1')
# axis[4].plot(np.array(time_string(times_vi_mms2)).astype("datetime64[ns]"),data_vi_mms2[:,2], 'green', label='MMS2')
# axis[4].plot(np.array(time_string(times_vi_mms3)).astype("datetime64[ns]"),data_vi_mms3[:,2], 'darkturquoise', label='MMS3')
# axis[4].plot(np.array(time_string(times_vi_mms4)).astype("datetime64[ns]"),data_vi_mms4[:,2], 'purple', label='MMS4')
# axis[4].legend(fontsize=14)
# axis[4].set_ylabel('$v_z$ [km/s]')



# # plot vertical lines for times to take the vx in neighboring msh
# for i in range(len(axis)):
#     for j in range(len(vx_times)):
#         axis[i].axvline(x = np.array(vx_times[j]).astype("datetime64[ns]"), color='r', linestyle='--')
# # plot vertical lines for times of density spikes
# for i in range(len(axis)):
#     for j in range(len(density_spikes_times)):
#         axis[i].axvline(x = np.array(density_spikes_times[j]).astype("datetime64[ns]"), color='k', linestyle='--')
        
# for i in range(len(axis)):
#     box = axis[i].get_position()
#     axis[i].set_position([box.x0, box.y0, box.width * 0.95, box.height])
#     axis[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))



# Find when all four SC encounter jumps in density

## Identifying a density spike at a time t, each SC will encounter this spike at slightly different times and at different positions. 

In [25]:

mms1_spike1_t1 = find_nearest(times_ni_mms1, spike1_st)
mms1_spike1_t2 = find_nearest(times_ni_mms1, spike1_end)
spike_mms1_idx1a = find_nearest(data_ni_mms1[mms1_spike1_t1:mms1_spike1_t2],spike1)
spike_mms1_idx1 = times_ni_mms1[mms1_spike1_t1:mms1_spike1_t2][spike_mms1_idx1a]


mms2_spike1_t1 = find_nearest(times_ni_mms2, spike1_st)
mms2_spike1_t2 = find_nearest(times_ni_mms2, spike1_end)
spike_mms2_idx1a = find_nearest(data_ni_mms2[mms2_spike1_t1:mms2_spike1_t2],spike1)
spike_mms2_idx1 = times_ni_mms2[mms2_spike1_t1:mms2_spike1_t2][spike_mms2_idx1a]

mms3_spike1_t1 = find_nearest(times_ni_mms3, spike1_st)
mms3_spike1_t2 = find_nearest(times_ni_mms3, spike1_end)
spike_mms3_idx1a = find_nearest(data_ni_mms3[mms3_spike1_t1:mms3_spike1_t2],spike1)
spike_mms3_idx1 = times_ni_mms3[mms3_spike1_t1:mms3_spike1_t2][spike_mms3_idx1a]


mms4_spike1_t1 = find_nearest(times_ni_mms4, spike1_st)
mms4_spike1_t2 = find_nearest(times_ni_mms4, spike1_end)
spike_mms4_idx1a = find_nearest(data_ni_mms4[mms4_spike1_t1:mms4_spike1_t2],spike1)
spike_mms4_idx1 = times_ni_mms4[mms4_spike1_t1:mms4_spike1_t2][spike_mms4_idx1a]
print('#####################')
print('density spike 1: ', spike1)
print('#####################')

print('spike_mms1_idx1',pd.to_datetime(spike_mms1_idx1,unit='s',origin='unix'))
print('spike_mms2_idx1',pd.to_datetime(spike_mms2_idx1,unit='s',origin='unix'))
print('spike_mms3_idx1',pd.to_datetime(spike_mms3_idx1,unit='s',origin='unix'))
print('spike_mms4_idx1',pd.to_datetime(spike_mms4_idx1,unit='s',origin='unix'))


print('mms1 closest density: ', data_ni_mms1[mms1_spike1_t1:mms1_spike1_t2][spike_mms1_idx1a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike1_t1:mms2_spike1_t2][spike_mms2_idx1a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike1_t1:mms2_spike1_t2][spike_mms2_idx1a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike1_t1:mms4_spike1_t2][spike_mms4_idx1a])


mms1_spike2_t1 = find_nearest(times_ni_mms1, spike2_st)
mms1_spike2_t2 = find_nearest(times_ni_mms1, spike2_end)
spike_mms1_idx2a = find_nearest(data_ni_mms1[mms1_spike2_t1:mms1_spike2_t2],spike2)
spike_mms1_idx2 = times_ni_mms1[mms1_spike2_t1:mms1_spike2_t2][spike_mms1_idx2a]


mms2_spike2_t1 = find_nearest(times_ni_mms2, spike2_st)
mms2_spike2_t2 = find_nearest(times_ni_mms2, spike2_end)
spike_mms2_idx2a = find_nearest(data_ni_mms2[mms2_spike2_t1:mms2_spike2_t2],spike2)
spike_mms2_idx2 = times_ni_mms2[mms2_spike2_t1:mms2_spike2_t2][spike_mms2_idx2a]

mms3_spike2_t1 = find_nearest(times_ni_mms3, spike2_st)
mms3_spike2_t2 = find_nearest(times_ni_mms3, spike2_end)
spike_mms3_idx2a = find_nearest(data_ni_mms3[mms3_spike2_t1:mms3_spike2_t2],spike2)
spike_mms3_idx2 = times_ni_mms3[mms3_spike2_t1:mms3_spike2_t2][spike_mms3_idx2a]


mms4_spike2_t1 = find_nearest(times_ni_mms4, spike2_st)
mms4_spike2_t2 = find_nearest(times_ni_mms4, spike2_end)
spike_mms4_idx2a = find_nearest(data_ni_mms4[mms4_spike2_t1:mms4_spike2_t2],spike2)
spike_mms4_idx2 = times_ni_mms4[mms4_spike2_t1:mms4_spike2_t2][spike_mms4_idx2a]
print('#####################')
print('density spike 2:', spike2)
print('#####################')
print('spike_mms1_idx2',pd.to_datetime(spike_mms1_idx2,unit='s',origin='unix'))
print('spike_mms2_idx2',pd.to_datetime(spike_mms2_idx2,unit='s',origin='unix'))
print('spike_mms3_idx2',pd.to_datetime(spike_mms3_idx2,unit='s',origin='unix'))
print('spike_mms4_idx2',pd.to_datetime(spike_mms4_idx2,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike2_t1:mms1_spike2_t2][spike_mms1_idx2a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike2_t1:mms2_spike2_t2][spike_mms2_idx2a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike2_t1:mms2_spike2_t2][spike_mms2_idx2a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike2_t1:mms4_spike2_t2][spike_mms4_idx2a])





mms1_spike3_t1 = find_nearest(times_ni_mms1, spike3_st)
mms1_spike3_t2 = find_nearest(times_ni_mms1, spike3_end)
spike_mms1_idx3a = find_nearest(data_ni_mms1[mms1_spike3_t1:mms1_spike3_t2],spike3)
spike_mms1_idx3 = times_ni_mms1[mms1_spike3_t1:mms1_spike3_t2][spike_mms1_idx3a]


mms2_spike3_t1 = find_nearest(times_ni_mms2, spike3_st)
mms2_spike3_t2 = find_nearest(times_ni_mms2, spike3_end)
spike_mms2_idx3a = find_nearest(data_ni_mms2[mms2_spike3_t1:mms2_spike3_t2],spike3)
spike_mms2_idx3 = times_ni_mms2[mms2_spike3_t1:mms2_spike3_t2][spike_mms2_idx3a]

mms3_spike3_t1 = find_nearest(times_ni_mms3, spike3_st)
mms3_spike3_t2 = find_nearest(times_ni_mms3, spike3_end)
spike_mms3_idx3a = find_nearest(data_ni_mms3[mms3_spike3_t1:mms3_spike3_t2],spike3)
spike_mms3_idx3 = times_ni_mms3[mms3_spike3_t1:mms3_spike3_t2][spike_mms3_idx3a]


mms4_spike3_t1 = find_nearest(times_ni_mms4, spike3_st)
mms4_spike3_t2 = find_nearest(times_ni_mms4, spike3_end)
spike_mms4_idx3a = find_nearest(data_ni_mms4[mms4_spike3_t1:mms4_spike3_t2],spike3)
spike_mms4_idx3 = times_ni_mms4[mms4_spike3_t1:mms4_spike3_t2][spike_mms4_idx3a]
print('#####################')
print('density spike 3:', spike3)
print('#####################')
print('spike_mms1_idx3',pd.to_datetime(spike_mms1_idx3,unit='s',origin='unix'))
print('spike_mms2_idx3',pd.to_datetime(spike_mms2_idx3,unit='s',origin='unix'))
print('spike_mms3_idx3',pd.to_datetime(spike_mms3_idx3,unit='s',origin='unix'))
print('spike_mms4_idx3',pd.to_datetime(spike_mms4_idx3,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike3_t1:mms1_spike3_t2][spike_mms1_idx3a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike3_t1:mms2_spike3_t2][spike_mms2_idx3a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike3_t1:mms2_spike3_t2][spike_mms2_idx3a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike3_t1:mms4_spike3_t2][spike_mms4_idx3a])





mms1_spike4_t1 = find_nearest(times_ni_mms1, spike4_st)
mms1_spike4_t2 = find_nearest(times_ni_mms1, spike4_end)
spike_mms1_idx4a = find_nearest(data_ni_mms1[mms1_spike4_t1:mms1_spike4_t2],spike4)
spike_mms1_idx4 = times_ni_mms1[mms1_spike4_t1:mms1_spike4_t2][spike_mms1_idx4a]


mms2_spike4_t1 = find_nearest(times_ni_mms2, spike4_st)
mms2_spike4_t2 = find_nearest(times_ni_mms2, spike4_end)
spike_mms2_idx4a = find_nearest(data_ni_mms2[mms2_spike4_t1:mms2_spike4_t2],spike4)
spike_mms2_idx4 = times_ni_mms2[mms2_spike4_t1:mms2_spike4_t2][spike_mms2_idx4a]

mms3_spike4_t1 = find_nearest(times_ni_mms3, spike4_st)
mms3_spike4_t2 = find_nearest(times_ni_mms3, spike4_end)
spike_mms3_idx4a = find_nearest(data_ni_mms3[mms3_spike4_t1:mms3_spike4_t2],spike4)
spike_mms3_idx4 = times_ni_mms3[mms3_spike4_t1:mms3_spike4_t2][spike_mms3_idx4a]


mms4_spike4_t1 = find_nearest(times_ni_mms4, spike4_st)
mms4_spike4_t2 = find_nearest(times_ni_mms4, spike4_end)
spike_mms4_idx4a = find_nearest(data_ni_mms4[mms4_spike4_t1:mms4_spike4_t2],spike4)
spike_mms4_idx4 = times_ni_mms4[mms4_spike4_t1:mms4_spike4_t2][spike_mms4_idx4a]
print('#####################')
print('density spike 4:', spike4)
print('#####################')
print('spike_mms1_idx4',pd.to_datetime(spike_mms1_idx4,unit='s',origin='unix'))
print('spike_mms2_idx4',pd.to_datetime(spike_mms2_idx4,unit='s',origin='unix'))
print('spike_mms3_idx4',pd.to_datetime(spike_mms3_idx4,unit='s',origin='unix'))
print('spike_mms4_idx4',pd.to_datetime(spike_mms4_idx4,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike4_t1:mms1_spike4_t2][spike_mms1_idx4a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike4_t1:mms2_spike4_t2][spike_mms2_idx4a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike4_t1:mms2_spike4_t2][spike_mms2_idx4a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike4_t1:mms4_spike4_t2][spike_mms4_idx4a])


mms1_spike5_t1 = find_nearest(times_ni_mms1, spike5_st)
mms1_spike5_t2 = find_nearest(times_ni_mms1, spike5_end)
spike_mms1_idx5a = find_nearest(data_ni_mms1[mms1_spike5_t1:mms1_spike5_t2],spike5)
spike_mms1_idx5 = times_ni_mms1[mms1_spike5_t1:mms1_spike5_t2][spike_mms1_idx5a]


mms2_spike5_t1 = find_nearest(times_ni_mms2, spike5_st)
mms2_spike5_t2 = find_nearest(times_ni_mms2, spike5_end)
spike_mms2_idx5a = find_nearest(data_ni_mms2[mms2_spike5_t1:mms2_spike5_t2],spike5)
spike_mms2_idx5 = times_ni_mms2[mms2_spike5_t1:mms2_spike5_t2][spike_mms2_idx5a]

mms3_spike5_t1 = find_nearest(times_ni_mms3, spike5_st)
mms3_spike5_t2 = find_nearest(times_ni_mms3, spike5_end)
spike_mms3_idx5a = find_nearest(data_ni_mms3[mms3_spike5_t1:mms3_spike5_t2],spike5)
spike_mms3_idx5 = times_ni_mms3[mms3_spike5_t1:mms3_spike5_t2][spike_mms3_idx5a]


mms4_spike5_t1 = find_nearest(times_ni_mms4, spike5_st)
mms4_spike5_t2 = find_nearest(times_ni_mms4, spike5_end)
spike_mms4_idx5a = find_nearest(data_ni_mms4[mms4_spike5_t1:mms4_spike5_t2],spike5)
spike_mms4_idx5 = times_ni_mms4[mms4_spike5_t1:mms4_spike5_t2][spike_mms4_idx5a]

print('#####################')
print('density spike 5:', spike5)
print('#####################')
print('spike_mms1_idx5',pd.to_datetime(spike_mms1_idx5,unit='s',origin='unix'))
print('spike_mms2_idx5',pd.to_datetime(spike_mms2_idx5,unit='s',origin='unix'))
print('spike_mms3_idx5',pd.to_datetime(spike_mms3_idx5,unit='s',origin='unix'))
print('spike_mms4_idx5',pd.to_datetime(spike_mms4_idx5,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike5_t1:mms1_spike5_t2][spike_mms1_idx5a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike5_t1:mms2_spike5_t2][spike_mms2_idx5a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike5_t1:mms2_spike5_t2][spike_mms2_idx5a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike5_t1:mms4_spike5_t2][spike_mms4_idx5a])

mms1_spike6_t1 = find_nearest(times_ni_mms1, spike6_st)
mms1_spike6_t2 = find_nearest(times_ni_mms1, spike6_end)
spike_mms1_idx6a = find_nearest(data_ni_mms1[mms1_spike6_t1:mms1_spike6_t2],spike6)
spike_mms1_idx6 = times_ni_mms1[mms1_spike6_t1:mms1_spike6_t2][spike_mms1_idx6a]


mms2_spike6_t1 = find_nearest(times_ni_mms2, spike6_st)
mms2_spike6_t2 = find_nearest(times_ni_mms2, spike6_end)
spike_mms2_idx6a = find_nearest(data_ni_mms2[mms2_spike6_t1:mms2_spike6_t2],spike6)
spike_mms2_idx6 = times_ni_mms2[mms2_spike6_t1:mms2_spike6_t2][spike_mms2_idx6a]

mms3_spike6_t1 = find_nearest(times_ni_mms3, spike6_st)
mms3_spike6_t2 = find_nearest(times_ni_mms3, spike6_end)
spike_mms3_idx6a = find_nearest(data_ni_mms3[mms3_spike6_t1:mms3_spike6_t2],spike6)
spike_mms3_idx6 = times_ni_mms3[mms3_spike6_t1:mms3_spike6_t2][spike_mms3_idx6a]


mms4_spike6_t1 = find_nearest(times_ni_mms4, spike6_st)
mms4_spike6_t2 = find_nearest(times_ni_mms4, spike6_end)
spike_mms4_idx6a = find_nearest(data_ni_mms4[mms4_spike6_t1:mms4_spike6_t2],spike6)
spike_mms4_idx6 = times_ni_mms4[mms4_spike6_t1:mms4_spike6_t2][spike_mms4_idx6a]
print('#####################')
print('density spike 6:', spike6)
print('#####################')
print('spike_mms1_idx6',pd.to_datetime(spike_mms1_idx6,unit='s',origin='unix'))
print('spike_mms2_idx6',pd.to_datetime(spike_mms2_idx6,unit='s',origin='unix'))
print('spike_mms3_idx6',pd.to_datetime(spike_mms3_idx6,unit='s',origin='unix'))
print('spike_mms4_idx6',pd.to_datetime(spike_mms4_idx6,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike6_t1:mms1_spike6_t2][spike_mms1_idx6a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike6_t1:mms2_spike6_t2][spike_mms2_idx6a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike6_t1:mms2_spike6_t2][spike_mms2_idx6a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike6_t1:mms4_spike6_t2][spike_mms4_idx6a])


mms1_spike7_t1 = find_nearest(times_ni_mms1, spike7_st)
mms1_spike7_t2 = find_nearest(times_ni_mms1, spike7_end)
spike_mms1_idx7a = find_nearest(data_ni_mms1[mms1_spike7_t1:mms1_spike7_t2],spike7)
spike_mms1_idx7 = times_ni_mms1[mms1_spike7_t1:mms1_spike7_t2][spike_mms1_idx7a]


mms2_spike7_t1 = find_nearest(times_ni_mms2, spike7_st)
mms2_spike7_t2 = find_nearest(times_ni_mms2, spike7_end)
spike_mms2_idx7a = find_nearest(data_ni_mms2[mms2_spike7_t1:mms2_spike7_t2],spike7)
spike_mms2_idx7 = times_ni_mms2[mms2_spike7_t1:mms2_spike7_t2][spike_mms2_idx7a]

mms3_spike7_t1 = find_nearest(times_ni_mms3, spike7_st)
mms3_spike7_t2 = find_nearest(times_ni_mms3, spike7_end)
spike_mms3_idx7a = find_nearest(data_ni_mms3[mms3_spike7_t1:mms3_spike7_t2],spike7)
spike_mms3_idx7 = times_ni_mms3[mms3_spike7_t1:mms3_spike7_t2][spike_mms3_idx7a]


mms4_spike7_t1 = find_nearest(times_ni_mms4, spike7_st)
mms4_spike7_t2 = find_nearest(times_ni_mms4, spike7_end)
spike_mms4_idx7a = find_nearest(data_ni_mms4[mms4_spike7_t1:mms4_spike7_t2],spike7)
spike_mms4_idx7 = times_ni_mms4[mms4_spike7_t1:mms4_spike7_t2][spike_mms4_idx7a]
print('#####################')
print('density spike 7:', spike7)
print('#####################')
print('spike_mms1_idx7',pd.to_datetime(spike_mms1_idx7,unit='s',origin='unix'))
print('spike_mms2_idx7',pd.to_datetime(spike_mms2_idx7,unit='s',origin='unix'))
print('spike_mms3_idx7',pd.to_datetime(spike_mms3_idx7,unit='s',origin='unix'))
print('spike_mms4_idx7',pd.to_datetime(spike_mms4_idx7,unit='s',origin='unix'))

print('mms1 closest density: ', data_ni_mms1[mms1_spike7_t1:mms1_spike7_t2][spike_mms1_idx7a])
print('mms2 closest density: ',data_ni_mms2[mms2_spike7_t1:mms2_spike7_t2][spike_mms2_idx7a])
print('mms3 closest density: ',data_ni_mms2[mms2_spike7_t1:mms2_spike7_t2][spike_mms2_idx7a])
print('mms4 closest density: ',data_ni_mms4[mms4_spike7_t1:mms4_spike7_t2][spike_mms4_idx7a])

#####################
density spike 1:  3.49
#####################
spike_mms1_idx1 2020-06-17 22:17:05.090184960
spike_mms2_idx1 2020-06-17 22:17:08.449000960
spike_mms3_idx1 2020-06-17 22:17:04.822615040
spike_mms4_idx1 2020-06-17 22:17:04.723514112
mms1 closest density:  3.5399668
mms2 closest density:  3.4828482
mms3 closest density:  3.4828482
mms4 closest density:  3.585058
#####################
density spike 2: 4.53
#####################
spike_mms1_idx2 2020-06-17 22:25:06.893629952
spike_mms2_idx2 2020-06-17 22:25:06.805792
spike_mms3_idx2 2020-06-17 22:25:06.776392960
spike_mms4_idx2 2020-06-17 22:25:06.825065984
mms1 closest density:  3.6490304
mms2 closest density:  3.4320414
mms3 closest density:  3.4320414
mms4 closest density:  3.5481184
#####################
density spike 3: 3.68
#####################
spike_mms1_idx3 2020-06-17 22:31:49.046510080
spike_mms2_idx3 2020-06-17 22:31:49.111515904
spike_mms3_idx3 2020-06-17 22:31:48.929541120
spike_mms4_idx3 2020-06-17 22:31:48

In [26]:

vx_times =['2020-06-17/22:17:29','2020-06-17/22:25:34','2020-06-17/22:32:03','2020-06-17/22:38:16',
'2020-06-17/22:44:25','2020-06-17/22:53:46','2020-06-17/23:01:31']

mms1_vx_spike1 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[0]).timestamp())
mms2_vx_spike1 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[0]).timestamp())
mms3_vx_spike1 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[0]).timestamp())
mms4_vx_spike1 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[0]).timestamp())

mms1_vx_spike2 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[1]).timestamp())
mms2_vx_spike2 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[1]).timestamp())
mms3_vx_spike2 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[1]).timestamp())
mms4_vx_spike2 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[1]).timestamp())

mms1_vx_spike3 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[2]).timestamp())
mms2_vx_spike3 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[2]).timestamp())
mms3_vx_spike3 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[2]).timestamp())
mms4_vx_spike3 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[2]).timestamp())

mms1_vx_spike4 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[3]).timestamp())
mms2_vx_spike4 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[3]).timestamp())
mms3_vx_spike4 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[3]).timestamp())
mms4_vx_spike4 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[3]).timestamp())

mms1_vx_spike5 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[4]).timestamp())
mms2_vx_spike5 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[4]).timestamp())
mms3_vx_spike5 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[4]).timestamp())
mms4_vx_spike5 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[4]).timestamp())


mms1_vx_spike6 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[5]).timestamp())
mms2_vx_spike6 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[5]).timestamp())
mms3_vx_spike6 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[5]).timestamp())
mms4_vx_spike6 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[5]).timestamp())

mms1_vx_spike7 = find_nearest(times_vi_mms1, pd.to_datetime(vx_times[6]).timestamp())
mms2_vx_spike7 = find_nearest(times_vi_mms2, pd.to_datetime(vx_times[6]).timestamp())
mms3_vx_spike7 = find_nearest(times_vi_mms3, pd.to_datetime(vx_times[6]).timestamp())
mms4_vx_spike7 = find_nearest(times_vi_mms4, pd.to_datetime(vx_times[6]).timestamp())


# vx_times =['2020-06-17 22:17:10','2020-06-17 22:25:34','2020-06-17 22:32:03','2020-06-17 22:38:16',
# '2020-06-17 22:44:25','2020-06-17 22:53:46','2020-06-17 23:01:31']
# #MSP times
# vx_times =['2020-06-17 22:16:36.000','2020-06-17 22:24:52.000','2020-06-17 22:31:36.000','2020-06-17 22:37:39.000',
#        '2020-06-17 22:43:53.000','2020-06-17 22:53:12.000', '2020-06-17 23:00:50']


# give a start time before the spike to begin searching the 4 MMS for that value
#spike1_st = pd.to_datetime('2020-06-17/22:16:59.000').timestamp()

In [27]:
# print(data_vi_mms1[mms1_vx_spike1,2])

### Find $V_{x} = \frac{\Delta x}{\Delta t}  $

In [28]:

# tt1 = [times_ni_mms1[mms1_spike1_t1:mms1_spike1_t2],times_ni_mms2[mms2_spike1_t1:mms2_spike1_t2],
#        times_ni_mms3[mms3_spike1_t1:mms3_spike1_t2],times_ni_mms4[mms4_spike1_t1:mms4_spike1_t2]]


# tt2 = [times_ni_mms1[mms1_spike2_t1:mms1_spike2_t2],times_ni_mms2[mms2_spike2_t1:mms2_spike2_t2],
#        times_ni_mms3[mms3_spike2_t1:mms3_spike2_t2],times_ni_mms4[mms4_spike2_t1:mms4_spike2_t2]]


# tt3 = [times_ni_mms1[mms1_spike3_t1:mms1_spike3_t2],times_ni_mms2[mms2_spike3_t1:mms2_spike3_t2],
#        times_ni_mms3[mms3_spike3_t1:mms3_spike3_t2],times_ni_mms4[mms4_spike3_t1:mms4_spike3_t2]]


# tt4 = [times_ni_mms1[mms1_spike4_t1:mms1_spike4_t2],times_ni_mms2[mms2_spike4_t1:mms2_spike4_t2],
#        times_ni_mms3[mms3_spike4_t1:mms3_spike4_t2],times_ni_mms4[mms4_spike4_t1:mms4_spike4_t2]]


# tt5 = [times_ni_mms1[mms1_spike5_t1:mms1_spike5_t2],times_ni_mms2[mms2_spike5_t1:mms2_spike5_t2],
#        times_ni_mms3[mms3_spike5_t1:mms3_spike5_t2],times_ni_mms4[mms4_spike5_t1:mms4_spike5_t2]]

# idx1 = [spike_mms1_idx1a,spike_mms2_idx1a,spike_mms3_idx1a,spike_mms4_idx1a]
# idx2 = [spike_mms1_idx2a,spike_mms2_idx2a,spike_mms3_idx2a,spike_mms4_idx2a]
# idx3 = [spike_mms1_idx3a,spike_mms2_idx3a,spike_mms3_idx3a,spike_mms4_idx3a]
# idx4 = [spike_mms1_idx4a,spike_mms2_idx4a,spike_mms3_idx4a,spike_mms4_idx4a]
# idx5 = [spike_mms1_idx5a,spike_mms2_idx5a,spike_mms3_idx5a,spike_mms4_idx5a]
# mec1 = [data_mec_mms1[mms1_spike1_t1:mms1_spike1_t2],data_mec_mms2[mms2_spike1_t1:mms2_spike1_t2],
#        data_mec_mms3[mms3_spike1_t1:mms3_spike1_t2],data_mec_mms4[mms4_spike1_t1:mms4_spike1_t2]]


# mec2 = [data_mec_mms1[mms1_spike2_t1:mms1_spike2_t2],data_mec_mms2[mms2_spike2_t1:mms2_spike2_t2],
#        data_mec_mms3[mms3_spike2_t1:mms3_spike2_t2],data_mec_mms4[mms4_spike2_t1:mms4_spike2_t2]]


# mec3 = [data_mec_mms1[mms1_spike3_t1:mms1_spike3_t2],data_mec_mms2[mms2_spike3_t1:mms2_spike3_t2],
#        data_mec_mms3[mms3_spike3_t1:mms3_spike3_t2],data_mec_mms4[mms4_spike3_t1:mms4_spike3_t2]]


# mec4 = [data_mec_mms1[mms1_spike4_t1:mms1_spike4_t2],data_mec_mms2[mms2_spike4_t1:mms2_spike4_t2],
#        data_mec_mms3[mms3_spike4_t1:mms3_spike4_t2],data_mec_mms4[mms4_spike4_t1:mms4_spike4_t2]]


# mec5 = [data_mec_mms1[mms1_spike5_t1:mms1_spike5_t2],data_mec_mms2[mms2_spike5_t1:mms2_spike5_t2],
#        data_mec_mms3[mms3_spike5_t1:mms3_spike5_t2],data_mec_mms4[mms4_spike5_t1:mms4_spike5_t2]]
# # Δt_cs1 = Δt_cs2 = Δt_cs3 = Δt_cs4 = Δt_cs5 = []
# # Δx_cs1 = Δx_cs2 = Δx_cs3 = Δx_cs4 = Δx_cs5 = []
# # apparently you have to specify each empty list individually because the lines above caused Δt_cs1,Δt_cs2, etc to all be the same values
# Δt_cs1 = []
# Δt_cs2 = []
# Δt_cs3 = []
# Δt_cs4 = []
# Δt_cs5 = []

# Δx_cs1 = []
# Δx_cs2 = []
# Δx_cs3 = []
# Δx_cs4 = []
# Δx_cs5 = []

# for i in range(4):
#     for j in range(4):
#         # if i != j:
#         Δt_cs1.append(np.abs(tt1[i][idx1[i]]- tt1[j][idx1[j]]))
#         Δt_cs2.append(np.abs(tt2[i][idx2[i]]- tt2[j][idx2[j]]))
#         Δt_cs3.append(np.abs(tt3[i][idx3[i]]- tt3[j][idx3[j]]))
#         Δt_cs4.append(np.abs(tt4[i][idx4[i]]- tt4[j][idx4[j]]))
#         Δt_cs5.append(np.abs(tt5[i][idx5[i]]- tt5[j][idx5[j]]))
        
#         Δx_cs1.append(np.abs(mec1[i][idx1[i],0]- mec1[j][idx1[j],0]))
#         Δx_cs2.append(np.abs(mec2[i][idx2[i],0]- mec2[j][idx2[j],0]))
#         Δx_cs3.append(np.abs(mec3[i][idx3[i],0]- mec3[j][idx3[j],0]))
#         Δx_cs4.append(np.abs(mec4[i][idx4[i],0]- mec4[j][idx4[j],0]))
#         Δx_cs5.append(np.abs(mec5[i][idx5[i],0]- mec5[j][idx5[j],0]))


# # Vx for all current sheets.
# Vx = np.mean([Δx_cs1,Δx_cs2,Δx_cs3,Δx_cs4,Δx_cs5])/np.mean([Δt_cs1,Δt_cs2,Δt_cs3,Δt_cs4,Δt_cs5])
# print('average Vx across all spikes:', np.mean([Δx_cs1,Δx_cs2,Δx_cs3,Δx_cs4,Δx_cs5])/np.mean([Δt_cs1,Δt_cs2,Δt_cs3,Δt_cs4,Δt_cs5]))


# x_cs = np.array([Δx_cs1,Δx_cs2,Δx_cs3,Δx_cs4,Δx_cs5])
# t_cs = np.array([Δt_cs1,Δt_cs2,Δt_cs3,Δt_cs4,Δt_cs5])

# for k in range(5):
#     print('Vx for density spike ',k,' = ',np.mean(x_cs[k])/np.mean(t_cs[k]),' [km/s] ')

# vi_mms1_spike1 = data_vi_mms1[mms1_spike1_t1:mms1_spike1_t2,0]
# vi_mms1_spike2 = data_vi_mms1[mms1_spike2_t1:mms1_spike2_t2,0]
# vi_mms1_spike3 = data_vi_mms1[mms1_spike3_t1:mms1_spike3_t2,0]
# vi_mms1_spike4 = data_vi_mms1[mms1_spike4_t1:mms1_spike4_t2,0]
# vi_mms1_spike5 = data_vi_mms1[mms1_spike5_t1:mms1_spike5_t2,0]

# vi_mms2_spike1 = data_vi_mms2[mms2_spike1_t1:mms2_spike1_t2,0]
# vi_mms2_spike2 = data_vi_mms2[mms2_spike2_t1:mms2_spike2_t2,0]
# vi_mms2_spike3 = data_vi_mms2[mms2_spike3_t1:mms2_spike3_t2,0]
# vi_mms2_spike4 = data_vi_mms2[mms2_spike4_t1:mms2_spike4_t2,0]
# vi_mms2_spike5 = data_vi_mms2[mms2_spike5_t1:mms2_spike5_t2,0]

# vi_mms3_spike1 = data_vi_mms3[mms3_spike1_t1:mms3_spike1_t2,0]
# vi_mms3_spike2 = data_vi_mms3[mms3_spike2_t1:mms3_spike2_t2,0]
# vi_mms3_spike3 = data_vi_mms3[mms3_spike3_t1:mms3_spike3_t2,0]
# vi_mms3_spike4 = data_vi_mms3[mms3_spike4_t1:mms3_spike4_t2,0]
# vi_mms3_spike5 = data_vi_mms3[mms3_spike5_t1:mms3_spike5_t2,0]

# vi_mms4_spike1 = data_vi_mms4[mms4_spike1_t1:mms4_spike1_t2,0]
# vi_mms4_spike2 = data_vi_mms4[mms4_spike2_t1:mms4_spike2_t2,0]
# vi_mms4_spike3 = data_vi_mms4[mms4_spike3_t1:mms4_spike3_t2,0]
# vi_mms4_spike4 = data_vi_mms4[mms4_spike4_t1:mms4_spike4_t2,0]
# vi_mms4_spike5 = data_vi_mms4[mms4_spike5_t1:mms4_spike5_t2,0]



# print('average Vix for all four spacecraft spike 1', np.mean([vi_mms1_spike1[spike_mms1_idx1a],vi_mms2_spike1[spike_mms2_idx1a],
#                                                               vi_mms3_spike1[spike_mms3_idx1a],vi_mms4_spike1[spike_mms4_idx1a]]))
# print('average Vix for all four spacecraft spike 2', np.mean([vi_mms1_spike2[spike_mms1_idx2a],vi_mms2_spike2[spike_mms2_idx2a],
#                                                               vi_mms3_spike2[spike_mms3_idx2a],vi_mms4_spike2[spike_mms4_idx2a]]))
# print('average Vix for all four spacecraft spike 3', np.mean([vi_mms1_spike3[spike_mms1_idx3a],vi_mms2_spike3[spike_mms2_idx3a],
#                                                               vi_mms3_spike3[spike_mms3_idx3a],vi_mms4_spike3[spike_mms4_idx3a]]))
# print('average Vix for all four spacecraft spike 4', np.mean([vi_mms1_spike4[spike_mms1_idx4a],vi_mms2_spike4[spike_mms2_idx4a],
#                                                               vi_mms3_spike4[spike_mms3_idx4a],vi_mms4_spike4[spike_mms4_idx4a]]))
# print('average Vix for all four spacecraft spike 5', np.mean([vi_mms1_spike5[spike_mms1_idx5a],vi_mms2_spike5[spike_mms2_idx5a],
#                                                               vi_mms3_spike5[spike_mms3_idx5a],vi_mms4_spike5[spike_mms4_idx5a]]))

# # also print out the actual velocity (Vx from FPI) to compare

# Find $\lambda_{KH} = \langle V_x \rangle \langle T \rangle $

Where $T = \frac{\Sigma T_i}{N}$

In [29]:
# # find when Bl transitions over 0
# transition_MMS1_bl1 = find_nearest(np.array(MMS1_B_LMN_CS1['Bl']),0)
# transition_MMS1_bl2 = find_nearest(np.array(MMS1_B_LMN_CS2['Bl']),0)
# transition_MMS1_bl3 = find_nearest(np.array(MMS1_B_LMN_CS3['Bl']),0)

# transition_MMS2_bl1 = find_nearest(np.array(MMS2_B_LMN_CS1['Bl']),0)
# transition_MMS2_bl2 = find_nearest(np.array(MMS2_B_LMN_CS2['Bl']),0)
# transition_MMS2_bl3 = find_nearest(np.array(MMS2_B_LMN_CS3['Bl']),0)

# transition_MMS3_bl1 = find_nearest(np.array(MMS3_B_LMN_CS1['Bl']),0)
# transition_MMS3_bl2 = find_nearest(np.array(MMS3_B_LMN_CS2['Bl']),0)
# transition_MMS3_bl3 = find_nearest(np.array(MMS3_B_LMN_CS3['Bl']),0)

# transition_MMS4_bl1 = find_nearest(np.array(MMS4_B_LMN_CS1['Bl']),0)
# transition_MMS4_bl2 = find_nearest(np.array(MMS4_B_LMN_CS2['Bl']),0)
# transition_MMS4_bl3 = find_nearest(np.array(MMS4_B_LMN_CS3['Bl']),0)

# # print(transition_MMS1_bl1)
# # print(transition_MMS1_bl2)
# #print(transition_MMS1_bl3)

# # # print(MMS1_B_LMN_CS1['Bl'][transition_MMS1_bl1])
# # # print(MMS1_B_LMN_CS2['Bl'][transition_MMS1_bl2])
# # print(MMS1_B_LMN_CS3['Bl'][transition_MMS1_bl3])
# # print(MMS2_B_LMN_CS3['Bl'][transition_MMS2_bl3])
# # print(MMS3_B_LMN_CS3['Bl'][transition_MMS3_bl3])
# # print(MMS4_B_LMN_CS3['Bl'][transition_MMS4_bl3])
# # # print(transition_MMS2_bl1)
# # # print(transition_MMS2_bl2)
# # # print(transition_MMS2_bl3)

# # # # print(MMS2_B_LMN_CS1['Bl'][transition_MMS2_bl1])
# # # # print(MMS2_B_LMN_CS2['Bl'][transition_MMS2_bl2])
# # # print(MMS2_B_LMN_CS3['Bl'][transition_MMS2_bl3])
# # print(transition_MMS1_bl3)
# # print(transition_MMS2_bl3)
# # print(transition_MMS3_bl3)
# # print(transition_MMS4_bl3)

## find the time in each of the SC where Bl transitions over 0

In [30]:

# CS1_Bl_0_t_MMS1 = MMS1_B_LMN_CS1['date'][transition_MMS1_bl1]
# CS2_Bl_0_t_MMS1 = MMS1_B_LMN_CS2['date'][transition_MMS1_bl2]
# CS3_Bl_0_t_MMS1 = MMS1_B_LMN_CS3['date'][transition_MMS1_bl3]

# CS1_Bl_0_t_MMS2 = MMS2_B_LMN_CS1['date'][transition_MMS2_bl1]
# CS2_Bl_0_t_MMS2 = MMS2_B_LMN_CS2['date'][transition_MMS2_bl2]
# CS3_Bl_0_t_MMS2 = MMS2_B_LMN_CS3['date'][transition_MMS2_bl3]

# CS1_Bl_0_t_MMS3 = MMS3_B_LMN_CS1['date'][transition_MMS3_bl1]
# CS2_Bl_0_t_MMS3 = MMS3_B_LMN_CS2['date'][transition_MMS3_bl2]
# CS3_Bl_0_t_MMS3 = MMS3_B_LMN_CS3['date'][transition_MMS3_bl3]

# CS1_Bl_0_t_MMS4 = MMS4_B_LMN_CS1['date'][transition_MMS4_bl1]
# CS2_Bl_0_t_MMS4 = MMS4_B_LMN_CS2['date'][transition_MMS4_bl2]
# CS3_Bl_0_t_MMS4 = MMS4_B_LMN_CS3['date'][transition_MMS4_bl3]

# MMS1_CS1 = MMS1_r_gse.date.searchsorted(CS1_Bl_0_t_MMS1)
# MMS2_CS1 = MMS2_r_gse.date.searchsorted(CS1_Bl_0_t_MMS2)
# MMS3_CS1 = MMS3_r_gse.date.searchsorted(CS1_Bl_0_t_MMS3)
# MMS4_CS1 = MMS4_r_gse.date.searchsorted(CS1_Bl_0_t_MMS4)

# MMS1_CS2 = MMS1_r_gse.date.searchsorted(CS2_Bl_0_t_MMS1)
# MMS2_CS2 = MMS2_r_gse.date.searchsorted(CS2_Bl_0_t_MMS2)
# MMS3_CS2 = MMS3_r_gse.date.searchsorted(CS2_Bl_0_t_MMS3)
# MMS4_CS2 = MMS4_r_gse.date.searchsorted(CS2_Bl_0_t_MMS4)

# MMS1_CS3 = MMS1_r_gse.date.searchsorted(CS3_Bl_0_t_MMS1)
# MMS2_CS3 = MMS2_r_gse.date.searchsorted(CS3_Bl_0_t_MMS2)
# MMS3_CS3 = MMS3_r_gse.date.searchsorted(CS3_Bl_0_t_MMS3)
# MMS4_CS3 = MMS4_r_gse.date.searchsorted(CS3_Bl_0_t_MMS4)


# print(MMS1_CS3)
# print(MMS2_CS3)
# print(MMS3_CS3)
# print(MMS4_CS3)
# print(MMS1_CS1)
# print(MMS2_CS1)
# print(MMS3_CS1)
# print(MMS4_CS1)

# print(CS3_Bl_0_t_MMS1)
# print(CS3_Bl_0_t_MMS2)
# print(CS3_Bl_0_t_MMS3)
# print(CS3_Bl_0_t_MMS4)

# a=np.ptp([CS3_Bl_0_t_MMS1, CS3_Bl_0_t_MMS2,CS3_Bl_0_t_MMS3, CS3_Bl_0_t_MMS4])

# print(a)

In [31]:

# print(MMS1_B_LMN_CS3['date'][transition_MMS1_bl3])
# print(MMS2_B_LMN_CS3['date'][transition_MMS2_bl3])
# print(MMS3_B_LMN_CS3['date'][transition_MMS3_bl3])
# print(MMS4_B_LMN_CS3['date'][transition_MMS4_bl3])



## SC position and time at Bl=0

In [32]:
# MMS1_CS1_t = MMS1_r_gse['times'][MMS1_CS1]
# MMS1_CS1_x = MMS1_r_gse['x'][MMS1_CS1]
# MMS2_CS1_t = MMS2_r_gse['times'][MMS2_CS1]
# MMS2_CS1_x = MMS2_r_gse['x'][MMS2_CS1]
# MMS3_CS1_t = MMS3_r_gse['times'][MMS3_CS1]
# MMS3_CS1_x = MMS3_r_gse['x'][MMS3_CS1]
# MMS4_CS1_t = MMS4_r_gse['times'][MMS4_CS1]
# MMS4_CS1_x = MMS4_r_gse['x'][MMS4_CS1]

# MMS1_CS2_t = MMS1_r_gse['times'][MMS1_CS2]
# MMS1_CS2_x = MMS1_r_gse['x'][MMS1_CS2]
# MMS2_CS2_t = MMS2_r_gse['times'][MMS2_CS2]
# MMS2_CS2_x = MMS2_r_gse['x'][MMS2_CS2]
# MMS3_CS2_t = MMS3_r_gse['times'][MMS3_CS2]
# MMS3_CS2_x = MMS3_r_gse['x'][MMS3_CS2]
# MMS4_CS2_t = MMS4_r_gse['times'][MMS4_CS2]
# MMS4_CS2_x = MMS4_r_gse['x'][MMS4_CS2]

# MMS1_CS3_t = MMS1_r_gse['times'][MMS1_CS3]
# MMS1_CS3_x = MMS1_r_gse['x'][MMS1_CS3]
# MMS2_CS3_t = MMS2_r_gse['times'][MMS2_CS3]
# MMS2_CS3_x = MMS2_r_gse['x'][MMS2_CS3]
# MMS3_CS3_t = MMS3_r_gse['times'][MMS3_CS3]
# MMS3_CS3_x = MMS3_r_gse['x'][MMS3_CS3]
# MMS4_CS3_t = MMS4_r_gse['times'][MMS4_CS3]
# MMS4_CS3_x = MMS4_r_gse['x'][MMS4_CS3]

# print("MMS1_CS1_t",MMS1_CS1_t)
# print("MMS1_CS1_x",MMS1_CS1_x)
# print("MMS2_CS1_t",MMS2_CS1_t)
# print("MMS2_CS1_x",MMS2_CS1_x)
# print("MMS3_CS1_t",MMS3_CS1_t)
# print("MMS3_CS1_x",MMS3_CS1_x)
# print("MMS4_CS1_t",MMS4_CS1_t)
# print("MMS4_CS1_x",MMS4_CS1_x)
# print("\n")

# print("MMS1_CS2_t",MMS1_CS2_t)
# print("MMS1_CS2_x",MMS1_CS2_x)
# print("MMS2_CS2_t",MMS2_CS2_t)
# print("MMS2_CS2_x",MMS2_CS2_x)
# print("MMS3_CS2_t",MMS3_CS2_t)
# print("MMS3_CS2_x",MMS3_CS2_x)
# print("MMS4_CS2_t",MMS4_CS2_t)
# print("MMS4_CS2_x",MMS4_CS2_x)
# print("\n")
# print("MMS1_CS3_t",MMS1_CS3_t)
# print("MMS1_CS3_x",MMS1_CS3_x)
# print("MMS2_CS3_t",MMS2_CS3_t)
# print("MMS2_CS3_x",MMS2_CS3_x)
# print("MMS3_CS3_t",MMS3_CS3_t)
# print("MMS3_CS3_x",MMS3_CS3_x)
# print("MMS4_CS3_t",MMS4_CS3_t)
# print("MMS4_CS3_x",MMS4_CS3_x)

## find Δt

In [33]:


# Δt_CS1 = np.ptp([MMS1_CS1_t, MMS2_CS1_t,MMS3_CS1_t, MMS4_CS1_t])
# Δx_CS1 = np.ptp([MMS1_CS1_x, MMS2_CS1_x,MMS3_CS1_x, MMS4_CS1_x])

# print('Δt_CS1',Δt_CS1)
# print('Δx_CS1',Δx_CS1)

# Δt_CS2 = np.ptp([MMS1_CS2_t, MMS2_CS2_t,MMS3_CS2_t, MMS4_CS2_t])
# Δx_CS2 = np.ptp([MMS1_CS2_x, MMS2_CS2_x,MMS3_CS2_x, MMS4_CS2_x])

# print('Δt_CS2',Δt_CS2)
# print('Δx_CS2',Δx_CS2)

# # not sure why all the CS3_t are the same here. when you print out the datetimes, there's a difference of .104808960
# Δt_CS3 = np.ptp([MMS1_CS3_t, MMS2_CS3_t,MMS3_CS3_t, MMS4_CS3_t])
# Δx_CS3 = np.ptp([MMS1_CS3_x, MMS2_CS3_x,MMS3_CS3_x, MMS4_CS3_x])

# print('Δt_CS3',Δt_CS3)
# print('Δx_CS3',Δx_CS3)

### Find $ V_x = \frac{\Delta x}{\Delta t}$

In [34]:
# Vx_CS1 =Δx_CS1/Δt_CS1

# print('Vx_CS1',Vx_CS1)

# Vx_CS2 =Δx_CS2/Δt_CS2
# print('Vx_CS2',Vx_CS2)

# Vx_CS3 =Δx_CS3/0.104808960
# print('Vx_CS3',Vx_CS3)

### Find period between vortices

In [35]:
# identified by eye using the following plots

In [36]:
# %matplotlib widget
# figure, axis = plt.subplots(2, 1,  figsize=(10, 5),sharex=True, constrained_layout=True) #constrained_layout=True
# #plt.subplots_adjust(wspace=0, hspace=0)
# # axis[0].plot(np.array(time_string(timeB)).astype("datetime64[ns]"),dataB[:,:])
# # axis[0].plot(MMS1_B_LMN_CS1['date'],MMS1_B_LMN_CS1['Bl'], 'k',label='$MMS1$')
# # axis[0].plot(MMS2_B_LMN_CS1['date'],MMS2_B_LMN_CS1['Bl'], 'green',label='$MMS2$')
# # axis[0].plot(MMS3_B_LMN_CS1['date'],MMS3_B_LMN_CS1['Bl'], 'darkturquoise',label='$MMS3$')
# # axis[0].plot(MMS4_B_LMN_CS1['date'],MMS4_B_LMN_CS1['Bl'], 'purple',label='$MMS4$')
# # axis[0].legend(fontsize=14)
# # axis[0].set_ylabel('$B_l$ $[nT]$')


# # axis[1].plot(MMS1_V_LMN_CS1['date'],MMS1_V_LMN_CS1['Vl'], 'k',label='$MMS1$')
# # axis[1].plot(MMS2_V_LMN_CS1['date'],MMS2_V_LMN_CS1['Vl'], 'green',label='$MMS2$')
# # axis[1].plot(MMS3_V_LMN_CS1['date'],MMS3_V_LMN_CS1['Vl'], 'darkturquoise',label='$MMS3$')
# # axis[1].plot(MMS4_V_LMN_CS1['date'],MMS4_V_LMN_CS1['Vl'], 'purple',label='$MMS4$')

# # axis[1].legend(fontsize=14)
# # axis[1].set_ylabel('$v_l$ $[km/s]$')



# # axis[0].plot(MMS1_B_LMN_CS2['date'],MMS1_B_LMN_CS2['Bl'], 'k',label='$MMS1$')
# # axis[0].plot(MMS2_B_LMN_CS2['date'],MMS2_B_LMN_CS2['Bl'], 'green',label='$MMS2$')
# # axis[0].plot(MMS3_B_LMN_CS2['date'],MMS3_B_LMN_CS2['Bl'], 'darkturquoise',label='$MMS3$')
# # axis[0].plot(MMS4_B_LMN_CS2['date'],MMS4_B_LMN_CS2['Bl'], 'purple',label='$MMS4$')
# # axis[0].legend(fontsize=14)
# # axis[0].set_ylabel('$B_l$ $[nT]$')


# # axis[1].plot(MMS1_V_LMN_CS2['date'],MMS1_V_LMN_CS2['Vl'], 'k',label='$MMS1$')
# # axis[1].plot(MMS2_V_LMN_CS2['date'],MMS2_V_LMN_CS2['Vl'], 'green',label='$MMS2$')
# # axis[1].plot(MMS3_V_LMN_CS2['date'],MMS3_V_LMN_CS2['Vl'], 'darkturquoise',label='$MMS3$')
# # axis[1].plot(MMS4_V_LMN_CS2['date'],MMS4_V_LMN_CS2['Vl'], 'purple',label='$MMS4$')


# axis[0].plot(MMS1_B_LMN_CS3['date'],MMS1_B_LMN_CS3['Bl'], 'k',label='$MMS1$')
# axis[0].plot(MMS2_B_LMN_CS3['date'],MMS2_B_LMN_CS3['Bl'], 'green',label='$MMS2$')
# axis[0].plot(MMS3_B_LMN_CS3['date'],MMS3_B_LMN_CS3['Bl'], 'darkturquoise',label='$MMS3$')
# axis[0].plot(MMS4_B_LMN_CS3['date'],MMS4_B_LMN_CS3['Bl'], 'purple',label='$MMS4$')
# axis[0].legend(fontsize=14)
# axis[0].set_ylabel('$B_l$ $[nT]$')


# axis[1].plot(MMS1_V_LMN_CS3['date'],MMS1_V_LMN_CS3['Vl'], 'k',label='$MMS1$')
# axis[1].plot(MMS2_V_LMN_CS3['date'],MMS2_V_LMN_CS3['Vl'], 'green',label='$MMS2$')
# axis[1].plot(MMS3_V_LMN_CS3['date'],MMS3_V_LMN_CS3['Vl'], 'darkturquoise',label='$MMS3$')
# axis[1].plot(MMS4_V_LMN_CS3['date'],MMS4_V_LMN_CS3['Vl'], 'purple',label='$MMS4$')

# # 17, 18, and 19

# # jets =['2020-06-17 23:42:40.000','2020-06-17 23:43:06.000','2020-06-17 23:49:18.000','2020-06-17 23:49:55.000',
# #        '2020-06-17 23:57:50.000','2020-06-17 23:58:33.000']
# # # plot vertical lines for start and stop time of jets. 
# # for i in range(len(axis)):
# #     for j in range(len(jets)):
# #         axis[i].axvline(x = np.array(jets[j]).astype("datetime64[ns]"), color='k', linestyle='--')



# # for i in range(len(axis)):
# #     box = axis[i].get_position()
# #     axis[i].set_position([box.x0, box.y0, box.width * 0.95, box.height])
# #     axis[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))



In [37]:
T_ni1 = np.mean([(spike5_t-spike4_t), (spike4_t-spike3_t), (spike3_t-spike2_t), (spike2_t-spike1_t)])# most accurately represents true period


print('Average period between density peaks (T):',T_ni1, '[s]')

Average period between density peaks (T): 405.0 [s]


# Method 2: Find Vx using MSH times instead

In [38]:

vx_spike1_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike1,0],data_vi_mms2[mms2_vx_spike1,0],data_vi_mms3[mms3_vx_spike1,0],
                        data_vi_mms4[mms4_vx_spike1,0]]))

vx_spike2_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike2,0],data_vi_mms2[mms2_vx_spike2,0],data_vi_mms3[mms3_vx_spike2,0],
                        data_vi_mms4[mms4_vx_spike2,0]]))

vx_spike3_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike3,0],data_vi_mms2[mms2_vx_spike3,0],data_vi_mms3[mms3_vx_spike3,0],
                        data_vi_mms4[mms4_vx_spike3,0]]))

vx_spike4_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike4,0],data_vi_mms2[mms2_vx_spike4,0],data_vi_mms3[mms3_vx_spike4,0],
                        data_vi_mms4[mms4_vx_spike4,0]]))

vx_spike5_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike5,0],data_vi_mms2[mms2_vx_spike5,0],data_vi_mms3[mms3_vx_spike5,0],
                        data_vi_mms4[mms4_vx_spike5,0]]))

vx_spike6_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike6,0],data_vi_mms2[mms2_vx_spike6,0],data_vi_mms3[mms3_vx_spike6,0],
                        data_vi_mms4[mms4_vx_spike6,0]]))

vx_spike7_ave = np.mean(np.abs([data_vi_mms1[mms1_vx_spike7,0],data_vi_mms2[mms2_vx_spike7,0],data_vi_mms3[mms3_vx_spike7,0],
                        data_vi_mms4[mms4_vx_spike7,0]]))


print('average Vx across all spikes:', np.mean([vx_spike1_ave,vx_spike1_ave,vx_spike2_ave,
                                                vx_spike3_ave,vx_spike1_ave,vx_spike4_ave,
                                                vx_spike5_ave,vx_spike1_ave,vx_spike6_ave,
                                                vx_spike7_ave]))
      

print('average Vix for all four spacecraft spike 1 ', vx_spike1_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 2 ', vx_spike2_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 3 ', vx_spike3_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 4 ', vx_spike4_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 5 ', vx_spike5_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 6 ', vx_spike6_ave, ' [km/s]' )
print('average Vix for all four spacecraft spike 7 ', vx_spike7_ave, ' [km/s]' )

print('λ_kh for density spike 1 ',(vx_spike1_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 2 ',(vx_spike2_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 3 ',(vx_spike3_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 4 ',(vx_spike4_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 5 ',(vx_spike5_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 6 ',(vx_spike6_ave)*T_ni1/6378,' [R_E]')
print('λ_kh for density spike 7 ',(vx_spike7_ave)*T_ni1/6378,' [R_E]')

λ_kh_ave = np.mean([vx_spike1_ave,vx_spike1_ave,vx_spike2_ave,
                                                vx_spike3_ave,vx_spike1_ave,vx_spike4_ave,
                                                vx_spike5_ave,vx_spike1_ave,vx_spike6_ave,
                                                vx_spike7_ave])*T_ni1/6378
      
print('average λ_kh for all density spikes ',λ_kh_ave,'[R_E]' )

average Vx across all spikes: 191.60358
average Vix for all four spacecraft spike 1  195.56793  [km/s]
average Vix for all four spacecraft spike 2  169.61392  [km/s]
average Vix for all four spacecraft spike 3  201.34412  [km/s]
average Vix for all four spacecraft spike 4  184.7429  [km/s]
average Vix for all four spacecraft spike 5  181.91148  [km/s]
average Vix for all four spacecraft spike 6  186.03304  [km/s]
average Vix for all four spacecraft spike 7  210.11853  [km/s]
λ_kh for density spike 1  12.418471701506277  [R_E]
λ_kh for density spike 2  10.770404273793032  [R_E]
λ_kh for density spike 3  12.78525667378954  [R_E]
λ_kh for density spike 4  11.731087549161149  [R_E]
λ_kh for density spike 5  11.551293653916998  [R_E]
λ_kh for density spike 6  11.813010236393811  [R_E]
λ_kh for density spike 7  13.34242783956447  [R_E]
average λ_kh for all density spikes  12.16673699394219 [R_E]


In [39]:
# λ_kh_4 = (Vx)*T_ni1

# print('average λ_kh :',(Vx)*T_ni1,'[km]')
# print('average λ_kh:',((Vx)*T_ni1)/6378,'[R_E]')
# print('\n')



# x_cs = np.array([Δx_cs1,Δx_cs2,Δx_cs3,Δx_cs4,Δx_cs5])
# t_cs = np.array([Δt_cs1,Δt_cs2,Δt_cs3,Δt_cs4,Δt_cs5])
###############################################################
# for k in range(5):
#     print('λ_kh for density spike ',k,' = ',(np.mean(x_cs[k])/np.mean(t_cs[k]))*T_ni1,' [km] ')
#     print('λ_kh for density spike ',k,' = ',((np.mean(x_cs[k])/np.mean(t_cs[k]))*T_ni1)/6378,' [R_E] ')
#     print('\n')

In [40]:
# 

# Get normal widths of each current sheet (CS) examined


## Find LMN components for all spacecraft

In [49]:

# # actual times of the start and stop of current sheet:

#times_temp =np.array([['2020-06-17/21:16:55.000','2020-06-17/21:17:07.000']]) #3 #Lentz et al CS #1
#times_temp =np.array([['2020-06-17/21:24:36.000','2020-06-17/21:25:20.000']]) #4 #Lentz et al CS #2
#times_temp =np.array([['2020-06-17/21:40:51.400','2020-06-17/21:41:20.799']]) #6 #Lentz et al CS #3
#times_temp = np.array([['2020-06-17/22:18:18.000','2020-06-17/22:18:28.900']]) #8 #Lentz et al CS #4
#times_temp =np.array([['2020-06-17/22:43:34','2020-06-17/22:44:03.599']]) #9 #Lentz et al CS #5
#times_temp =np.array([['2020-06-17/22:46:25.250','2020-06-17/22:46:38.500']])#10 #Lentz et al CS #6
#times_temp =np.array([['2020-06-17/23:03:50.000','2020-06-17/23:03:58.000']]) #11 #Lentz et al CS #7
#times_temp = np.array([['2020-06-17/23:10:43.000','2020-06-17/23:10:52.00']]) #13 #Lentz et al CS #8
#times_temp =np.array([['2020-06-17/23:17:17.299','2020-06-17/23:17:28.000']]) #14 #Lentz et al CS #9
#times_temp =np.array([['2020-06-17/23:34:55.400','2020-06-17/23:35:17.500']]) #15 #Lentz et al CS #10
#times_temp =np.array([['2020-06-17/23:42:47.000','2020-06-17/23:42:59.000']]) #17 #Lentz et al CS #11
#times_temp =np.array([['2020-06-17/23:49:26.400','2020-06-17/23:49:46.350']]) #18 #Lentz et al CS #12
#times_temp =np.array([['2020-06-17/23:57:57.599','2020-06-17/23:58:25.000']]) #19 #Lentz et al CS #13
#times_temp =np.array([['2020-06-18/00:09:41.000','2020-06-18/00:10:05.000']]) #21 #Lentz et al CS #14
# # #suggestion to move start time from 00:22:14.000 to 00:22:09.000
#times_temp =np.array([['2020-06-18/00:22:09.000','2020-06-18/00:22:35.599']]) #22 #Lentz et al CS #15

# jet_candidate_timeranges = np.array([['2020-06-17/21:16:55.000','2020-06-17/21:17:07.000'], #3 wonderful example
# ['2020-06-17/21:24:36.000','2020-06-17/21:25:20.000'], #4 
# ['2020-06-17/21:40:51.400','2020-06-17/21:41:20.799']]) #6 

# jet_candidate_timeranges = np.array([['2020-06-17/22:18:18.000','2020-06-17/22:18:28.900'], #8 
# ['2020-06-17/22:43:34','2020-06-17/22:44:03.599'], #9 2 merging jets
# ['2020-06-17/22:46:25.250','2020-06-17/22:46:38.500']]) #10

# jet_candidate_timeranges = np.array([['2020-06-17/23:03:50.000','2020-06-17/23:03:58.000'], #11  great example
# ['2020-06-17/23:10:43.000','2020-06-17/23:10:52.00'], #13 
# ['2020-06-17/23:17:17.299','2020-06-17/23:17:28.000']]) #14 

# jet_candidate_timeranges = np.array([['2020-06-17/23:34:55.400','2020-06-17/23:35:17.500'], # 15 
# ['2020-06-17/23:42:47.000','2020-06-17/23:42:59.000'], #17 
# ['2020-06-17/23:49:26.400','2020-06-17/23:49:46.350']]) #18 

jet_candidate_timeranges = np.array([['2020-06-17/23:57:57.599','2020-06-17/23:58:25.000'], #19 
['2020-06-18/00:09:41.000','2020-06-18/00:10:05.000'], #21
['2020-06-18/00:22:09.000','2020-06-18/00:22:35.599']]) #22
















# # jet_candidate_timeranges = np.array([['2020-06-17/21:16:50.000','2020-06-17/21:17:10.000'], #3 wonderful example
# # ['2020-06-17/21:24:34.000','2020-06-17/21:25:22.000'], #4 
# # ['2020-06-17/21:40:49.000','2020-06-17/21:41:23.000']]) #6 

# # jet_candidate_timeranges = np.array([['2020-06-17/22:18:18.200','2020-06-17/22:18:31.000'], #8 
# # ['2020-06-17/22:43:33.000','2020-06-17/22:44:05.000'], #9 2 merging jets
# # ['2020-06-17/22:46:24.000','2020-06-17/22:46:39.000']]) #10

# # jet_candidate_timeranges = np.array([['2020-06-17/23:03:49.000','2020-06-17/23:03:59.000'], #11  great example
# # ['2020-06-17/23:10:42.000','2020-06-17/23:10:53.000'], #13 
# # ['2020-06-17/23:17:16.000','2020-06-17/23:17:29.000']]) #14 

# jet_candidate_timeranges = np.array([['2020-06-17/23:34:54.000','2020-06-17/23:35:18.000'], # 15 
# ['2020-06-17/23:42:45.000','2020-06-17/23:43:02.000'], #17 
# ['2020-06-17/23:49:23.000','2020-06-17/23:49:49.000']]) #18 

# # jet_candidate_timeranges = np.array([['2020-06-17/23:57:55.000','2020-06-17/23:58:27.000'], #19 
# # ['2020-06-18/00:09:41.000','2020-06-18/00:10:05.000'], #21
# # ['2020-06-18/00:22:08.000','2020-06-18/00:22:36.000']]) #22

mag_arr_str = ['MMS1_B_gse.csv','MMS2_B_gse.csv','MMS3_B_gse.csv','MMS4_B_gse.csv']
v_arr_str = ['MMS1_Vi.csv','MMS2_Vi.csv','MMS3_Vi.csv','MMS4_Vi.csv']
n_arr_str = ['MMS1_ni.csv','MMS2_ni.csv','MMS3_ni.csv','MMS4_ni.csv']

for i in range(len(jet_candidate_timeranges)):
    # create an array to hold L x,y,z GSE and N x,y,z GSE values to plot
    L_vec = np.empty((5,3))
    N_vec = np.empty((5,3))
    L_vec[0,:]=[0,0,0] # create 0 vector for use in plotting 
    N_vec[0,:]=[0,0,0] # create 0 vector for use in plotting 
    for m in range(len(mag_arr_str)):
        mag_arr = pd.read_csv(mag_arr_str[m],header = 0,names = ['times','Bx','By','Bz','Bmag','date'],parse_dates=['date'])
        v_arr = pd.read_csv(v_arr_str[m],header = 0,names = ['times','Vix','Viy','Viz','date'],parse_dates=['date'])
        n_arr = pd.read_csv(n_arr_str[m],header = 0,names = ['times','ni','date'],parse_dates=['date'])
        # create empty lists to store all of the L M N coordinates in
        all_l = np.empty((len(jet_candidate_timeranges[:,0]),3))
        all_m = np.empty((len(jet_candidate_timeranges[:,0]),3))
        all_n = np.empty((len(jet_candidate_timeranges[:,0]),3))
        
        dtb = pd.Timedelta(hours = 0, minutes = 0, seconds = 2)
    
        m1 = pd.to_datetime(jet_candidate_timeranges[i,0])
        m2 = pd.to_datetime(jet_candidate_timeranges[i,1])
    
        time_b = np.array(mag_arr['times'])
        bx = np.array(pd.to_numeric(mag_arr['Bx']))
        by = np.array(pd.to_numeric(mag_arr['By']))
        bz = np.array(pd.to_numeric(mag_arr['Bz']))
    
        time_v = np.array(v_arr['times'])
        vx = np.array(pd.to_numeric(v_arr['Vix']))
        vy = np.array(pd.to_numeric(v_arr['Viy']))
        vz = np.array(pd.to_numeric(v_arr['Viz']))
        # t5_v = v_arr.date.searchsorted(m1)
        # t6_v = v_arr.date.searchsorted(m2)
        # uncomment to include data during dtb time:
        t5_v = v_arr.date.searchsorted(m1-dtb)
        t6_v = v_arr.date.searchsorted(m2+dtb)
        # print(t5_v)
        # velocity from specified start (t5) and stop(t6) time of current sheet
        vx_t5_t6 = vx[t5_v:t6_v]
        vy_t5_t6 = vy[t5_v:t6_v]
        vz_t5_t6 = vz[t5_v:t6_v]
        time_v_t5_t6 = time_v[t5_v:t6_v]
    
        # search the B-field array date column for m1
        # t5_b = mag_arr.date.searchsorted(m1)
        # t6_b = mag_arr.date.searchsorted(m2)
        # uncomment to include data during dtb time:
        t5_b = mag_arr.date.searchsorted(m1 - dtb)
        t6_b = mag_arr.date.searchsorted(m2 + dtb)
    
        # B-field from specified start (t5) and stop (t6) time of current sheet
        bx_t5_t6 = bx[t5_b:t6_b]
        by_t5_t6 = by[t5_b:t6_b]
        bz_t5_t6 = bz[t5_b:t6_b]
        time_b_t5_t6 = time_b[t5_b:t6_b]
    
        # Find Eigenvalues and Eigen vectors from MVAB analysis
        # print('#########################')
        # print('Current sheet '+jet_candidate_timeranges[i,0]+' - '+ jet_candidate_timeranges[i,1])
        # print('MMS'+str(m+1)+':')
        # print('#########################')
        E_vals,E_vec = mms_mvab(m1,m2,mag_arr_str[m])
        lc,mc,nc,lv,mv,nv,v1a,v2a = Cross_product_normal(E_vals,E_vec,jet_candidate_timeranges[i,0],
                                                  jet_candidate_timeranges[i,1],mag_arr_str[m])
        
        # print('L:',lc)
        # print('M:',mc)
        # print('N:',nc)
        # print('\n')

    #TEST A
        all_l[i,:] = lv
        all_m[i,:] = mv
        all_n[i,:] = nv

        bn = []
        bl = []
        bm = []
        vn = []
        vl = []
        vm = []
        # for each B-value at each time stamp, project B into LMN coordinates.
        for t in range(len(time_b_t5_t6)):
            bvx_temp = bx_t5_t6[t]
            bvy_temp = by_t5_t6[t]
            bvz_temp = bz_t5_t6[t]
            # create matrix where columns are the x,y,z component at that timestamp to be used in the following dot product
            bvec = np.stack((bvx_temp,bvy_temp,bvz_temp), axis=-1)
            # find dot product between bvec and nc
            bl.append(np.dot(bvec,lc))
            bm.append(np.dot(bvec,mc))
            bn.append(np.dot(bvec,nc))
        # for each v value at each t, project into LMN coordinate system    
        for t in range(len(time_v_t5_t6)):
            # calculate cross product normals using eigen values, eigen vectors, and the start and stop 
            # time for each jet candidate. 
            vx_temp = vx_t5_t6[t]
            vy_temp = vy_t5_t6[t]
            vz_temp = vz_t5_t6[t]
            # create matrix where columns are the x,y,z component at that timestamp to be used in the following dot product
            Vvec = np.stack((vx_temp,vy_temp,vz_temp), axis=-1)
            vl.append(np.dot(Vvec,lc))
            vm.append(np.dot(Vvec,mc))
            vn.append(np.dot(Vvec,nc))


        #print('Vl',vl)
    
        # create a unique string to name files/images based on the start time of the timerange
        string1 = np.array2string(jet_candidate_timeranges[i,0])
        string2 = np.array2string(jet_candidate_timeranges[i,1])
        #print(string)
        time_period_string1 = string1.replace(":", "_")
        time_period_string1 = time_period_string1.replace("/", "_")
        time_period_string1 = time_period_string1.replace("T", "_")
        time_period_string1 = time_period_string1.replace("'", "_")
        #print( time_period_string)
        # time string has .000 or .000000000 remove them for saving
        if len(time_period_string1) > 19:
            time_period_string1 = time_period_string1[0:20]
            #print(string)
    
        time_period_string2 = string2.replace(":", "_")
        time_period_string2 = time_period_string2.replace("/", "_")
        time_period_string2 = time_period_string2.replace("T", "_")
        time_period_string2 = time_period_string2.replace("'", "_")
        if len(time_period_string2) > 19:
            time_period_string2 = time_period_string2[0:20]
            #print(string)
        time_period_string = time_period_string1 + time_period_string2
    
        ################################################
        # Generate B_{LMN} and V_{LMN} data structures!
        ################################################
        if m == 0:
            MMS1_B_LMN = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn})
            MMS1_V_LMN = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS1_n = pd.DataFrame({"times": time_v_t5_t6, "n": n_arr['ni'][t5_v:t6_v]})
            MMS1_B_LMN['date'] = pd.to_datetime(MMS1_B_LMN['times'],unit='s',origin='unix')
            MMS1_V_LMN['date'] = pd.to_datetime(MMS1_V_LMN['times'],unit='s',origin='unix')
            MMS1_n['date'] = pd.to_datetime(MMS1_n['times'],unit='s',origin='unix')
            MMS1_L = np.array([[0,0],[lc[0],lc[1]]]) # save the x and y gse component of the L vector
            MMS1_N = np.array([[0,0],[nc[0],nc[1]]]) 

           
        if m == 1:
            MMS2_B_LMN = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn})
            MMS2_V_LMN = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS2_n = pd.DataFrame({"times": time_v_t5_t6, "n": n_arr['ni'][t5_v:t6_v]})
            MMS2_B_LMN['date'] = pd.to_datetime(MMS2_B_LMN['times'],unit='s',origin='unix')
            MMS2_V_LMN['date'] = pd.to_datetime(MMS2_V_LMN['times'],unit='s',origin='unix')
            MMS2_n['date'] = pd.to_datetime(MMS2_n['times'],unit='s',origin='unix')
            MMS2_L = np.array([[0,0],[lc[0],lc[1]]]) # save the x and y gse component of the L vector
            MMS2_N = np.array([[0,0],[nc[0],nc[1]]]) 
            
        if m == 2:
            MMS3_B_LMN = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn})
            MMS3_V_LMN = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS3_n = pd.DataFrame({"times": time_v_t5_t6, "n": n_arr['ni'][t5_v:t6_v]})
            MMS3_B_LMN['date'] = pd.to_datetime(MMS3_B_LMN['times'],unit='s',origin='unix')
            MMS3_V_LMN['date'] = pd.to_datetime(MMS3_V_LMN['times'],unit='s',origin='unix')
            MMS3_n['date'] = pd.to_datetime(MMS3_n['times'],unit='s',origin='unix')       
            MMS3_L = np.array([[0,0],[lc[0],lc[1]]]) # save the x and y gse component of the L vector
            MMS3_N = np.array([[0,0],[nc[0],nc[1]]]) 
            
        if m == 3:
            MMS4_B_LMN = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,
                                "Bm": bm, "Bn": bn})
            MMS4_V_LMN = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,
                                "Vm": vm, "Vn": vn})
            MMS4_n = pd.DataFrame({"times": time_v_t5_t6, "n": n_arr['ni'][t5_v:t6_v]})
            MMS4_B_LMN['date'] = pd.to_datetime(MMS4_B_LMN['times'],unit='s',origin='unix')
            MMS4_V_LMN['date'] = pd.to_datetime(MMS4_V_LMN['times'],unit='s',origin='unix')
            MMS4_n['date'] = pd.to_datetime(MMS4_n['times'],unit='s',origin='unix')   
            MMS4_L = np.array([[0,0],[lc[0],lc[1]]]) # save the x and y gse component of the L vector
            MMS4_N = np.array([[0,0],[nc[0],nc[1]]]) 

    ################################################
    # Generate B_{LMN} and V_{LMN} data structures!
    ################################################

#     print(len(time_v_t5_t6))

        if i==0 and m==0:
            MMS1_B_LMN_CS1 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS1_B_LMN_CS1['date'] = pd.to_datetime(MMS1_B_LMN_CS1['times'],unit='s',origin='unix')
            MMS1_V_LMN_CS1 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS1_V_LMN_CS1['date'] = pd.to_datetime(MMS1_V_LMN_CS1['times'],unit='s',origin='unix')
        if i==0 and m==1:
            MMS2_B_LMN_CS1 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS2_B_LMN_CS1['date'] = pd.to_datetime(MMS2_B_LMN_CS1['times'],unit='s',origin='unix')
            MMS2_V_LMN_CS1 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS2_V_LMN_CS1['date'] = pd.to_datetime(MMS2_V_LMN_CS1['times'],unit='s',origin='unix')
        if i==0 and m==2:    
            MMS3_B_LMN_CS1 = pd.DataFrame({"times":time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS3_B_LMN_CS1['date'] = pd.to_datetime(MMS3_B_LMN_CS1['times'],unit='s',origin='unix')
            MMS3_V_LMN_CS1 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS3_V_LMN_CS1['date'] = pd.to_datetime(MMS3_V_LMN_CS1['times'],unit='s',origin='unix')
        if i==0 and m==3:   
            MMS4_B_LMN_CS1 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS4_B_LMN_CS1['date'] = pd.to_datetime(MMS4_B_LMN_CS1['times'],unit='s',origin='unix')
            MMS4_V_LMN_CS1 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS4_V_LMN_CS1['date'] = pd.to_datetime(MMS4_V_LMN_CS1['times'],unit='s',origin='unix')
            
        if i==1 and m==0:
            MMS1_B_LMN_CS2 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS1_B_LMN_CS2['date'] = pd.to_datetime(MMS1_B_LMN_CS2['times'],unit='s',origin='unix')
            MMS1_V_LMN_CS2 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS1_V_LMN_CS2['date'] = pd.to_datetime(MMS1_V_LMN_CS2['times'],unit='s',origin='unix')
        if i==1 and m==1:
            MMS2_B_LMN_CS2 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS2_B_LMN_CS2['date'] = pd.to_datetime(MMS2_B_LMN_CS2['times'],unit='s',origin='unix')
            MMS2_V_LMN_CS2 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS2_V_LMN_CS2['date'] = pd.to_datetime(MMS2_V_LMN_CS2['times'],unit='s',origin='unix')
        if i==1 and m==2:    
            MMS3_B_LMN_CS2 = pd.DataFrame({"times":time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS3_B_LMN_CS2['date'] = pd.to_datetime(MMS3_B_LMN_CS2['times'],unit='s',origin='unix')
            MMS3_V_LMN_CS2 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS3_V_LMN_CS2['date'] = pd.to_datetime(MMS3_V_LMN_CS2['times'],unit='s',origin='unix')
        if i==1 and m==3:   
            MMS4_B_LMN_CS2 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS4_B_LMN_CS2['date'] = pd.to_datetime(MMS4_B_LMN_CS2['times'],unit='s',origin='unix')
            MMS4_V_LMN_CS2 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS4_V_LMN_CS2['date'] = pd.to_datetime(MMS4_V_LMN_CS2['times'],unit='s',origin='unix')        
        if i==2 and m==0:
            MMS1_B_LMN_CS3 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS1_B_LMN_CS3['date'] = pd.to_datetime(MMS1_B_LMN_CS3['times'],unit='s',origin='unix')
            MMS1_V_LMN_CS3 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS1_V_LMN_CS3['date'] = pd.to_datetime(MMS1_V_LMN_CS3['times'],unit='s',origin='unix')
        if i==2 and m==1:
            MMS2_B_LMN_CS3 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS2_B_LMN_CS3['date'] = pd.to_datetime(MMS2_B_LMN_CS3['times'],unit='s',origin='unix')
            MMS2_V_LMN_CS3 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS2_V_LMN_CS3['date'] = pd.to_datetime(MMS2_V_LMN_CS3['times'],unit='s',origin='unix')
        if i==2 and m==2:    
            MMS3_B_LMN_CS3 = pd.DataFrame({"times":time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS3_B_LMN_CS3['date'] = pd.to_datetime(MMS3_B_LMN_CS3['times'],unit='s',origin='unix')
            MMS3_V_LMN_CS3 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS3_V_LMN_CS3['date'] = pd.to_datetime(MMS3_V_LMN_CS3['times'],unit='s',origin='unix')
        if i==2 and m==3:   
            MMS4_B_LMN_CS3 = pd.DataFrame({"times": time_b_t5_t6, "Bl": bl,"Bm": bm, "Bn": bn}) 
            MMS4_B_LMN_CS3['date'] = pd.to_datetime(MMS4_B_LMN_CS3['times'],unit='s',origin='unix')
            MMS4_V_LMN_CS3 = pd.DataFrame({"times": time_v_t5_t6, "Vl": vl,"Vm": vm, "Vn": vn})
            MMS4_V_LMN_CS3['date'] = pd.to_datetime(MMS4_V_LMN_CS3['times'],unit='s',origin='unix')


In [50]:
dtv = pd.Timedelta(hours = 0, minutes = 0, seconds = 1)

V_LMN_CS = [MMS2_V_LMN_CS1,MMS2_V_LMN_CS2,MMS2_V_LMN_CS3]

for i in range(len(jet_candidate_timeranges)):
    m1 = pd.to_datetime(jet_candidate_timeranges[i,0])
    m2 = pd.to_datetime(jet_candidate_timeranges[i,1])
    
    idx_at_cs_st = V_LMN_CS[i].date.searchsorted(m1)
    idx_b4_cs_st = V_LMN_CS[i].date.searchsorted(m1-dtv)
    
    idx_at_cs_end = V_LMN_CS[i].date.searchsorted(m2)
    idx_after_cs_end = V_LMN_CS[i].date.searchsorted(m2+dtv)
    
    #VN1 is the average VN for a short time period (say 1 s or so?) right up against the CS start time
    VN1 = np.mean(np.array(V_LMN_CS[i]['Vn'][idx_b4_cs_st:idx_at_cs_st]))
    
    
    # same average right after the end of the CS for about 1 s too
    VN2 = np.mean(np.array(V_LMN_CS[i]['Vn'][idx_at_cs_end:idx_after_cs_end]))
    
    # duration in seconds
    # DURATION OF THE CURRENT SHEET??
    string1 = np.array2string(jet_candidate_timeranges[i,0])
    string2 = np.array2string(jet_candidate_timeranges[i,1])
    time_period_string1 = string1.replace("/", " ")
    time_period_string1 = time_period_string1.replace("'", "")
    time_period_string2 = string2.replace("/", " ")
    time_period_string2 = time_period_string2.replace("'", "")
    dt = (np.array(time_period_string2).astype("datetime64[s]") - np.array(time_period_string1).astype("datetime64[s]"))/np.timedelta64(1,'s')
    
    
    
    
    
    #We need that CS normal width in terms of km and ion inertial length (diavg). Here, the average ion inertial length:
    
    m1 = pd.to_datetime(jet_candidate_timeranges[i,0])
    m2 = pd.to_datetime(jet_candidate_timeranges[i,1])
    
    npavg1 = np.mean(np.array(MMS2_ni['ni'][MMS2_ni.date.searchsorted(m1-dtv):MMS2_ni.date.searchsorted(m1)]))
    
    npavg2 = np.mean(np.array(MMS2_ni['ni'][MMS2_ni.date.searchsorted(m2):MMS2_ni.date.searchsorted(m2+dtv)]))
    
    
    normal_widths = dt *(VN1+VN2)/2

   
    
    # average plasma density m-3
    npavg = ((npavg1 + npavg2 )/2.0)*1e6  
    
    
    #ion angular frequency (Hz)
    omega_pi = np.sqrt(npavg*q**2/(mp*eps0))  
    
    
    
    # average ion inertial length (km)
    diavg = (c/omega_pi)*1e-3  
    
    print(m1,'-',m2)
    print('CS normal width [km]: ',normal_widths)
    print('CS normal width in terms of ion inertial length:',normal_widths/diavg)
    print('npavg [cm-3]',npavg/1e6)
    print('omega_pi [Hz]:',omega_pi)
    print('diavg [km]:',diavg)
    print('\n')
    
    
#print('$np_{ave} = $',"%.2f" %  npavg/1e6)



#display(Latex(f'$np_(avg) [cm^(-3)]$ {npavg/1e6:.2f}')) 
   # display(Latex(f'$m_c \cdot l_c =$ {np.dot(mc,lc):.2f}'))
   # display(Latex(f'$n_c \cdot m_c =$ {np.dot(nc,mc):.2f}'))


2020-06-17 23:57:57.599000 - 2020-06-17 23:58:25
CS normal width [km]:  1575.9111563682525
CS normal width in terms of ion inertial length: 8.773967862619768
npavg [cm-3] 1.6072497367858887
omega_pi [Hz]: 1669.1102041625072
diavg [km]: 179.61214139867047


2020-06-18 00:09:41 - 2020-06-18 00:10:05
CS normal width [km]:  2928.0907717561604
CS normal width in terms of ion inertial length: 13.65372988217528
npavg [cm-3] 1.1274266242980957
omega_pi [Hz]: 1397.9365946330881
diavg [km]: 214.45354471079253


2020-06-18 00:22:09 - 2020-06-18 00:22:35.599000
CS normal width [km]:  1567.7530681114051
CS normal width in terms of ion inertial length: 7.401764973284625
npavg [cm-3] 1.155766248703003
omega_pi [Hz]: 1415.3972076434295
diavg [km]: 211.8080044111013


